In [1]:
pip install tensorflow-gpu

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: astunparse>=1.6.0 in c:\users\bolit\anaconda3\lib\site-packages (from tensorflow-gpu) (1.6.3)



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import roc_curve, auc,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
from matplotlib import gridspec
from scipy import stats, interpolate
import os
from tensorflow.keras import backend as K  

import tqdm


import math

os.environ["CUDA_VISIBLE_DEVICES"]="2"
tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)

per_process_gpu_memory_fraction: 0.333

## dataset read and processing

In [3]:
original_data=pd.read_hdf("/Users/bolit/OneDrive/Documentos/Tec/7_semestre/investigacion/CWoaLaSimulation/content/events_anomalydetection_v2.features.h5")
features_sig=original_data.query("label == 1")
features_bg=original_data.query("label == 0")


features_bg2=pd.read_hdf("/Users/bolit/OneDrive/Documentos/Tec/7_semestre/investigacion/CWoaLaSimulation/content/events_anomalydetection_DelphesHerwig_qcd_features.h5")

In [4]:
features_sig

,pxj1,pyj1,pzj1,mj1,tau1j1,tau2j1,tau3j1,pxj2,pyj2,pzj2,mj2,tau1j2,tau2j2,tau3j2,label
1000000,-1698.670044,-884.039978,723.843018,105.035004,83.721703,46.282101,13.635600,1539.439941,372.238007,-295.865997,461.574005,431.343994,52.344799,37.284901,1.0
1000001,1246.660034,-1133.010010,-921.987000,159.865997,133.781998,58.968601,30.377399,-1218.489990,1108.380005,182.147003,514.883972,462.654999,138.789001,67.805801,1.0
1000002,420.975006,-1739.790039,281.553986,93.665802,77.925797,10.605900,6.916510,-510.779999,1484.069946,227.175995,475.316986,217.113998,29.424000,21.020300,1.0
1000003,161.048996,-1664.859985,-2005.099976,116.327003,61.819698,38.143600,18.414400,-188.942993,1556.900024,-561.664001,561.236023,348.179993,102.625000,53.422699,1.0
1000004,-564.754028,-1315.599976,-1087.410034,513.015991,276.446991,50.629799,35.460999,326.164001,1050.239990,1201.000000,108.752998,89.666603,40.928699,17.055799,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099995,1069.660034,659.874023,218.751007,126.183998,122.486000,27.608700,17.924801,-956.169006,-297.311005,-2204.350098,108.889999,21.177299,10.582400,9.138600,1.0
1099996,-1286.619995,-86.162498,-1366.270020,115.719002,109.853996,29.830200,22.489201,1145.729980,136.792007,1216.780029,489.053009,416.747009,84.599998,66.767502,1.0
1099997,-149.330002,1781.459961,-58.690899,508.045013,495.290985,82.283600,43.567902,84.726700,-1378.569946,-1485.469971,91.104897,79.120102,46.537300,23.227301,1.0
1099998,1584.699951,-731.156982,-196.348007,114.938004,83.769897,12.898200,9.031230,-1515.079956,783.245972,498.704010,553.737000,366.188995,192.139008,81.398201,1.0


In [5]:
alpha = 0.1

def load_dataT21(input_frame):
    dataset = input_frame[["mj1","tau1j1","tau2j1","mj2","tau1j2","tau2j2"]]
    dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
    dataset["mjTwo"] = dataset[["mj1", "mj2"]].max(axis=1)
    dataset["mjOne"] = dataset[["mj1", "mj2"]].min(axis=1)

    dataset["tau1jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau1jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau21jOne"] = dataset["tau2jOne"]/dataset["tau1jOne"]
    dataset["tau21jTwo"] = dataset["tau2jTwo"]/dataset["tau1jTwo"]
    dataset["mjTwo"] = dataset["mjTwo"]/1000. + 2*alpha*dataset["mjj"]
    dataset["mjOne"] = dataset["mjOne"]/1000. + alpha*dataset["mjj"]
    dataset["mjDelta"] = (dataset["mjTwo"] - dataset["mjOne"])

    dataset = dataset.fillna(0)
    dataset = dataset[["mjj","mjOne","mjDelta", "tau21jOne", "tau21jTwo"]]
    return dataset.to_numpy()

def load_dataT32(input_frame):
    dataset = input_frame[["mj1","tau1j1","tau2j1","mj2","tau1j2","tau2j2","tau3j1","tau3j2"]]
    dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
    dataset["mjTwo"] = dataset[["mj1", "mj2"]].max(axis=1)
    dataset["mjOne"] = dataset[["mj1", "mj2"]].min(axis=1)

    dataset["tau2jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau2jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau3jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau3jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau32jOne"] = dataset["tau3jOne"]/dataset["tau2jOne"]
    dataset["tau32jTwo"] = dataset["tau3jTwo"]/dataset["tau2jTwo"] 
    dataset["mjTwo"] = dataset["mjTwo"]/1000. + 2*alpha*dataset["mjj"]
    dataset["mjOne"] = dataset["mjOne"]/1000. + alpha*dataset["mjj"]
    dataset["mjDelta"] = (dataset["mjTwo"] - dataset["mjOne"])

    dataset = dataset.fillna(0)
    dataset = dataset[["mjj","mjOne","mjDelta", "tau32jOne", "tau32jTwo"]]
    return dataset.to_numpy()

def load_data_more(input_frame):
    dataset = input_frame[["mj1","tau1j1","tau2j1","mj2","tau1j2","tau2j2","tau3j1","tau3j2"]]
    
    #Masa ya al cuadrado
    dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
    
    
    dataset["mjTwo"] = dataset[["mj1", "mj2"]].max(axis=1)
    dataset["mjOne"] = dataset[["mj1", "mj2"]].min(axis=1)

    dataset["tau1jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau1jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau1j2"]
    dataset["tau2jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau2j2"]
    dataset["tau21jOne"] = dataset["tau2jOne"]/dataset["tau1jOne"]
    dataset["tau21jTwo"] = dataset["tau2jTwo"]/dataset["tau1jTwo"]
    
    #tau32
    dataset["tau3jOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau3jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau3j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau3j2"]
    dataset["tau32jOne"] = dataset["tau3jOne"]/dataset["tau2jOne"]
    dataset["tau32jTwo"] = dataset["tau3jTwo"]/dataset["tau2jTwo"]   
    
    #pT
    dataset["pT1"] = (input_frame["pxj1"]**2+input_frame["pyj1"]**2)**0.5/1000.
    dataset["pT2"] = (input_frame["pxj2"]**2+input_frame["pyj2"]**2)**0.5/1000.
    dataset["pTjOne"] = (dataset["mjOne"] == dataset["mj1"])*dataset["pT1"]+(dataset["mjOne"] == dataset["mj2"])*dataset["pT2"]
    dataset["pTjTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["pT1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["pT2"]    
    
    #eta
    dataset["E1"] = ((1000*dataset["pT1"])**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5/1000.
    dataset["E2"] = ((1000*dataset["pT2"])**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5/1000.
    dataset["y1"] = 0.5*np.log((dataset["E1"] + input_frame["pzj1"]/1000.)/(dataset["E1"] - input_frame["pzj1"]/1000.))
    dataset["y2"] = 0.5*np.log((dataset["E2"] + input_frame["pzj2"]/1000.)/(dataset["E2"] - input_frame["pzj2"]/1000.))
    dataset["Dy"] = dataset["y1"]-dataset["y2"]
    
    dataset["mjTwo"] = dataset["mjTwo"]/1000. + 2*alpha*dataset["mjj"]
    dataset["mjOne"] = dataset["mjOne"]/1000. + alpha*dataset["mjj"]
    dataset["mjDelta"] = (dataset["mjTwo"] - dataset["mjOne"])

    dataset = dataset.fillna(0)
    dataset = dataset[["mjj","mjOne","mjDelta", "tau21jOne", "tau21jTwo", "tau32jOne", "tau32jTwo","pTjOne","pTjTwo","Dy"]]
    return dataset.to_numpy()

column_labelsT21 = [
    r"$m_{JJ}$",
    r"$m^{J_1}$",
    "$m^{J_2} - m^{J_1}$",
    r"$\tau_{2,1}^{J_1}$", 
    r"$\tau_{2,1}^{J_2}$",
]

column_labelsT32 = [
    r"$m_{JJ}$",
    r"$m^{J_1}$",
    "$m^{J_2} - m^{J_1}$",
    r"$\tau_{3,2}^{J_1}$", 
    r"$\tau_{3,2}^{J_2}$",
]

In [6]:
dataset_bgT21=load_dataT21(features_bg)
dataset_bg2T21=load_dataT21(features_bg2)
dataset_sigT21=load_dataT21(features_sig)

dataset_bgT32=load_dataT32(features_bg)
dataset_bg2T32=load_dataT32(features_bg2)
dataset_sigT32=load_dataT32(features_sig)

C:\Users\bolit\AppData\Local\Temp\ipykernel_18460\259476259.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
C:\Users\bolit\AppData\Local\Temp\ipykernel_18460\259476259.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

C:\Users\bolit\AppData\Local\Temp\ipykernel_18460\259476259.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["tau1jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau1j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau1j2"]
C:\Users\bolit\AppData\Local\Temp\ipykernel_18460\259476259.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["tau2jTwo"] = (dataset["mjTwo"] == dataset["mj1"])*dataset["tau2j1"]+(dataset["mjTwo"] == dataset["mj2"])*dataset["tau2j2"]
C:\Users\bolit\AppData

In [7]:
dataset_bg_more=load_data_more(features_bg)
dataset_bg2_more=load_data_more(features_bg2)
dataset_sig_more=load_data_more(features_sig)

C:\Users\bolit\AppData\Local\Temp\ipykernel_18460\259476259.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["mjj"] = (((input_frame["pxj1"]**2+input_frame["pyj1"]**2+input_frame["pzj1"]**2+input_frame["mj1"]**2)**0.5+(input_frame["pxj2"]**2+input_frame["pyj2"]**2+input_frame["pzj2"]**2+input_frame["mj2"]**2)**0.5)**2-(input_frame["pxj1"]+input_frame["pxj2"])**2-(input_frame["pyj1"]+input_frame["pyj2"])**2-(input_frame["pzj1"]+input_frame["pzj2"])**2)**0.5/1000.
C:\Users\bolit\AppData\Local\Temp\ipykernel_18460\259476259.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

## data preparation CWoLa T_21

making it machine-learnable, setting params for ML methods, setting signal significance

In [8]:
from sklearn.model_selection import train_test_split

SR_low = 3.3
SR_high = 3.7
test_size = 0.5
EPOCHS=20
batch_size = 200

def sr_crit(d):
    return (d[:,0] < SR_high) & (d[:,0] >= SR_low)

bg_srsbT21, bg2_srsbT21, sig_srsbT21 = sr_crit(dataset_bgT21), sr_crit(dataset_bg2T21), sr_crit(dataset_sigT21)

SR_background_dataT21 = dataset_bgT21[bg_srsbT21]
SB_background_dataT21 = dataset_bgT21[~bg_srsbT21]
print(~bg_srsbT21)

SR_background_simT21 = dataset_bg2T21[bg2_srsbT21]
SB_background_simT21 = dataset_bg2T21[~bg2_srsbT21]

SR_signalT21 = dataset_sigT21[sig_srsbT21]
SB_signalT21 = dataset_sigT21[~sig_srsbT21]

N_inputs = len(SR_background_dataT21.T) - 1


((SR_background_dataT21, SR_background_data_fortestT21),
 (SR_background_simT21, SR_background_sim_fortestT21),
 (SB_background_dataT21, SB_background_data_fortestT21),
 (SB_background_simT21, SB_background_sim_fortestT21),
 ) = [train_test_split(arr, test_size=test_size) for arr in [
    SR_background_dataT21, SR_background_simT21,
    SB_background_dataT21, SB_background_simT21,
]]

mn,mx = np.percentile(np.concatenate([SB_background_dataT21, SB_background_simT21, SR_background_dataT21, 
                                      SR_background_simT21]), [1,99], axis=0)

def norm_func(d):
    return (d - mn)/(mx - mn)

Nsig = 1000
Nsig_SB = int(np.round(len(SB_signalT21)*Nsig/len(SR_signalT21)))

fmt = '{:>20}:  {}'
print(fmt.format('# signal events', Nsig))
print(fmt.format('signal significance', len(SR_signalT21[0:Nsig])/len(SR_background_dataT21)**0.5))
print(fmt.format('s/b ratio', len(SR_signalT21[0:Nsig])/len(SR_background_dataT21)))

n_injectionsT21 = 20
SR_signals_to_injectT21 = [None] * n_injectionsT21
SB_signals_to_injectT21 = [None] * n_injectionsT21
signals_to_testT21 = [None] * n_injectionsT21

for i in range(n_injectionsT21):
    idx = np.isin(range(len(SR_signalT21)), np.random.choice(SR_signalT21.shape[0], size=Nsig, replace=False))
    SR_signals_to_injectT21[i] = SR_signalT21[idx, :]
    signals_to_testT21[i] = SR_signalT21[~idx, :]
    
    idx = np.isin(range(len(SB_signalT21)), np.random.choice(SB_signalT21.shape[0], size=Nsig_SB, replace=False))
    SB_signals_to_injectT21[i] = SB_signalT21[idx, :]

[False  True  True ...  True  True  True]
     # signal events:  1000
 signal significance:  4.059677443835412
           s/b ratio:  0.016480980947986026


## CWoLa T_21

With relu activation

In [9]:
#CWoLa with T21 and relu 
cwola_rocsT21 = pd.DataFrame(columns=['fpr', 'tpr'])
SB_width = 0.3

for i in tqdm.tqdm(range(n_injectionsT21)):
    
    SR_signal_to_injectT21 = SR_signals_to_injectT21[i]
    SB_signal_to_injectT21 = SB_signals_to_injectT21[i]

    signal_to_testT21 = signals_to_testT21[i]
        
    X_cwola_trainT21 = np.concatenate([SR_signal_to_injectT21, SR_background_dataT21, SB_signal_to_injectT21, SB_background_dataT21])
    Y_cwola_trainT21 = np.concatenate([np.ones(len(SR_signal_to_injectT21)), np.ones(len(SR_background_dataT21)),
                                    np.zeros(len(SB_signal_to_injectT21)), np.zeros(len(SB_background_dataT21))])
    
    X_cwola_trainT21, Y_cwola_trainT21 = shuffle(X_cwola_trainT21, Y_cwola_trainT21, )

    sbmjjT21 = X_cwola_trainT21[:,0]
    sb_critT21 = ((sbmjjT21 > SR_low - SB_width)&(sbmjjT21 <= SR_high + SB_width))
    
    X_cwola_trainT21 = X_cwola_trainT21[sb_critT21]
    Y_cwola_trainT21 = Y_cwola_trainT21[sb_critT21]
    
    w_lowT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] <= SR_low).sum())
    w_highT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] >= SR_high).sum())

    W_cwola_trainT21 = np.ones_like(Y_cwola_trainT21)
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] <= SR_low)] = w_lowT21
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] >= SR_high)] = w_highT21
    
    X_cwola_valT21 = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
    Y_cwola_valT21 = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
    
    K.clear_session()
    model_cwolaT21 = Sequential()
    model_cwolaT21.add(Dense(64, input_dim=N_inputs, activation='relu')) 
    model_cwolaT21.add(Dense(64, activation='relu'))
    model_cwolaT21.add(Dense(64, activation='relu'))
    model_cwolaT21.add(Dense(1, activation='sigmoid'))
    model_cwolaT21.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_cwolaT21 = model_cwolaT21.fit(X_cwola_trainT21[:,1:], Y_cwola_trainT21, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0, sample_weight=W_cwola_trainT21)

    scores_cwolaT21 = model_cwolaT21.predict(X_cwola_valT21[:,1:],batch_size=batch_size)
        
    fpr_cwolaT21, tpr_cwolaT21, _ = roc_curve(Y_cwola_valT21, scores_cwolaT21)
    cwola_rocsT21 = cwola_rocsT21.append({'fpr': fpr_cwolaT21, 'tpr': tpr_cwolaT21}, ignore_index=True)

  0%|                                                                                           | 0/20 [00:04<?, ?it/s]


KeyboardInterrupt: 

## data preparation CWoLa T_32

making it machine-learnable, setting params for ML methods, setting signal significance

In [ ]:
SR_low = 3.3
SR_high = 3.7
test_size = 0.5
EPOCHS=20
batch_size = 200

def sr_crit(d):
    return (d[:,0] < SR_high) & (d[:,0] >= SR_low)

bg_srsbT32, bg2_srsbT32, sig_srsbT32 = sr_crit(dataset_bgT32), sr_crit(dataset_bg2T32), sr_crit(dataset_sigT32)

SR_background_dataT32 = dataset_bgT32[bg_srsbT32]
SB_background_dataT32 = dataset_bgT32[~bg_srsbT32]

SR_background_simT32 = dataset_bg2T32[bg2_srsbT32]
SB_background_simT32 = dataset_bg2T32[~bg2_srsbT32]

SR_signalT32 = dataset_sigT32[sig_srsbT32]
SB_signalT32 = dataset_sigT32[~sig_srsbT32]

N_inputs = len(SR_background_dataT32.T) - 1


((SR_background_dataT32, SR_background_data_fortestT32),
 (SR_background_simT32, SR_background_sim_fortestT32),
 (SB_background_dataT32, SB_background_data_fortestT32),
 (SB_background_simT32, SB_background_sim_fortestT32),
 ) = [train_test_split(arr, test_size=test_size) for arr in [
    SR_background_dataT32, SR_background_simT32,
    SB_background_dataT32, SB_background_simT32,
]]

mn,mx = np.percentile(np.concatenate([SB_background_dataT32, SB_background_simT32, SR_background_dataT32, 
                                      SR_background_simT32]), [1,99], axis=0)

def norm_func(d):
    return (d - mn)/(mx - mn)

Nsig = 1000
Nsig_SB = int(np.round(len(SB_signalT32)*Nsig/len(SR_signalT32)))

fmt = '{:>20}:  {}'
print(fmt.format('# signal events', Nsig))
print(fmt.format('signal significance', len(SR_signalT32[0:Nsig])/len(SR_background_dataT32)**0.5))
print(fmt.format('s/b ratio', len(SR_signalT32[0:Nsig])/len(SR_background_dataT32)))

n_injectionsT32 = 20
SR_signals_to_injectT32 = [None] * n_injectionsT32
SB_signals_to_injectT32 = [None] * n_injectionsT32
signals_to_testT32 = [None] * n_injectionsT32

for i in range(n_injectionsT32):
    idx = np.isin(range(len(SR_signalT32)), np.random.choice(SR_signalT32.shape[0], size=Nsig, replace=False))
    SR_signals_to_injectT32[i] = SR_signalT32[idx, :]
    signals_to_testT32[i] = SR_signalT32[~idx, :]
    
    idx = np.isin(range(len(SB_signalT32)), np.random.choice(SB_signalT32.shape[0], size=Nsig_SB, replace=False))
    SB_signals_to_injectT32[i] = SB_signalT32[idx, :]

## CWoLa T_32

With relu activation

In [ ]:
#CWoLa with T32 and relu
cwola_rocsT32 = pd.DataFrame(columns=['fpr', 'tpr'])
SB_width = 0.3

for i in tqdm.tqdm(range(n_injectionsT32)):
    
    SR_signal_to_injectT32 = SR_signals_to_injectT32[i]
    SB_signal_to_injectT32 = SB_signals_to_injectT32[i]

    signal_to_testT32 = signals_to_testT32[i]
        
    X_cwola_trainT32 = np.concatenate([SR_signal_to_injectT32, SR_background_dataT32, SB_signal_to_injectT32, SB_background_dataT32])
    Y_cwola_trainT32 = np.concatenate([np.ones(len(SR_signal_to_injectT32)), np.ones(len(SR_background_dataT32)),
                                    np.zeros(len(SB_signal_to_injectT32)), np.zeros(len(SB_background_dataT32))])
    
    X_cwola_trainT32, Y_cwola_trainT32 = shuffle(X_cwola_trainT32, Y_cwola_trainT32, )

    sbmjjT32 = X_cwola_trainT32[:,0]
    sb_critT32 = ((sbmjjT32 > SR_low - SB_width)&(sbmjjT32 <= SR_high + SB_width))
    
    X_cwola_trainT32 = X_cwola_trainT32[sb_critT32]
    Y_cwola_trainT32 = Y_cwola_trainT32[sb_critT32]
    
    w_lowT32 = 0.5*(len(SR_signal_to_injectT32) + len(SR_background_dataT32))/((X_cwola_trainT32[:,0] <= SR_low).sum())
    w_highT32 = 0.5*(len(SR_signal_to_injectT32) + len(SR_background_dataT32))/((X_cwola_trainT32[:,0] >= SR_high).sum())

    W_cwola_trainT32 = np.ones_like(Y_cwola_trainT32)
    W_cwola_trainT32[(Y_cwola_trainT32 == 0) & (X_cwola_trainT32[:,0] <= SR_low)] = w_lowT32
    W_cwola_trainT32[(Y_cwola_trainT32 == 0) & (X_cwola_trainT32[:,0] >= SR_high)] = w_highT32
    
    X_cwola_valT32 = np.concatenate([signal_to_testT32,SR_background_data_fortestT32])
    Y_cwola_valT32 = np.concatenate([np.ones(len(signal_to_testT32)),np.zeros(len(SR_background_data_fortestT32))])
    
    K.clear_session()
    model_cwolaT32 = Sequential()
    model_cwolaT32.add(Dense(64, input_dim=N_inputs, activation='relu')) 
    model_cwolaT32.add(Dense(64, activation='relu'))
    model_cwolaT32.add(Dense(64, activation='relu'))
    model_cwolaT32.add(Dense(1, activation='sigmoid'))
    model_cwolaT32.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_cwolaT32 = model_cwolaT32.fit(X_cwola_trainT32[:,1:], Y_cwola_trainT32, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0, sample_weight=W_cwola_trainT32)

    scores_cwolaT32 = model_cwolaT32.predict(X_cwola_valT32[:,1:],batch_size=batch_size)
        
    fpr_cwolaT32, tpr_cwolaT32, _ = roc_curve(Y_cwola_valT32, scores_cwolaT32)
    cwola_rocsT32 = cwola_rocsT32.append({'fpr': fpr_cwolaT32, 'tpr': tpr_cwolaT32}, ignore_index=True)

In [ ]:
 with np.errstate(divide='ignore'):
    tpr_pts = np.linspace(0, 1, 10000)
    
    # CWoLa T21
    fpr_interp = [None] * len(cwola_rocsT21)

    for i, row in cwola_rocsT21.iterrows():
        fpr, tpr = row
        interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

        fpr_pts = interp(tpr_pts)
        fpr_interp[i] = fpr_pts

    fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

    cwola_maxT21 = np.nanmax(fpr_interp, axis=0).data
    cwola_minT21 = np.nanmin(fpr_interp, axis=0).data
    cwola_medT21 = np.median(fpr_interp, axis=0).data
    cwola_meanT21 = np.nanmean(fpr_interp, axis=0).data
    cwola_stdT21 = np.nanstd(fpr_interp, axis=0).data
    
    sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
    cwola_smeanT21 = np.nanmean(sig_interp, axis=0).data
    cwola_sstdT21 = np.nanstd(sig_interp, axis=0).data
    
    
    # CWoLa T32
    fpr_interp = [None] * len(cwola_rocsT32)

    for i, row in cwola_rocsT32.iterrows():
        fpr, tpr = row
        interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

        fpr_pts = interp(tpr_pts)
        fpr_interp[i] = fpr_pts

    fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

    cwola_maxT32 = np.nanmax(fpr_interp, axis=0).data
    cwola_minT32 = np.nanmin(fpr_interp, axis=0).data
    cwola_medT32 = np.median(fpr_interp, axis=0).data
    cwola_meanT32 = np.nanmean(fpr_interp, axis=0).data
    cwola_stdT32 = np.nanstd(fpr_interp, axis=0).data
    
    sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
    cwola_smeanT32 = np.nanmean(sig_interp, axis=0).data
    cwola_sstdT32 = np.nanstd(sig_interp, axis=0).data

## Plotting

In [ ]:
bands = {'cwola_T21': (cwola_maxT21, cwola_medT21, cwola_minT21, cwola_meanT21, cwola_stdT21, cwola_smeanT21, cwola_sstdT21, {'label':'$T_{21}$', 'color':'blue'}),
         'cwola_T32': (cwola_maxT32, cwola_medT32, cwola_minT32, cwola_meanT32, cwola_stdT32, cwola_smeanT32, cwola_sstdT32, {'label':'$T_{32}$', 'color':'red'}),}

In [ ]:
# Plot without banding
mylambda=.5
myklambda = 1 #np.round(len(SR_background_data)/len(SR_background_sim), 3)

fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1])
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.yscale("log")
plt.ylim([1,5*1e4])
ERR=False
#plt.plot(tpr_sup,1./fpr_sup,label="Supervised",color='green')

for band in bands:
    if band in ['old-sacwola', 'hl-sacwola']:
        continue
    mu,std = bands[band][3:5]

    plt.plot(tpr_pts, mu, **bands[band][-1])
    if ERR:
        plt.fill_between(tpr_pts, mu - std, mu + std, alpha=.2, color=bands[band][-1]['color'])
#plt.plot(tpr_sup,1./tpr_sup,color="black",ls=":",label="Random")

plt.xlabel(r'Signal Efficiency (True Positive Rate)',fontsize=20)
plt.ylabel(r"Rejection (1/False Positive Rate)",fontsize=20)
plt.legend(frameon=False,fontsize=15,loc="upper right")

With T_21 is better

## selu activation

In [ ]:
#CWoLa with T21 and selu 
cwola_rocsT21_selu = pd.DataFrame(columns=['fpr', 'tpr'])
SB_width = 0.3

for i in tqdm.tqdm(range(n_injectionsT21)):
    
    SR_signal_to_injectT21 = SR_signals_to_injectT21[i]
    SB_signal_to_injectT21 = SB_signals_to_injectT21[i]

    signal_to_testT21 = signals_to_testT21[i]
        
    X_cwola_trainT21 = np.concatenate([SR_signal_to_injectT21, SR_background_dataT21, SB_signal_to_injectT21, SB_background_dataT21])
    Y_cwola_trainT21 = np.concatenate([np.ones(len(SR_signal_to_injectT21)), np.ones(len(SR_background_dataT21)),
                                    np.zeros(len(SB_signal_to_injectT21)), np.zeros(len(SB_background_dataT21))])
    
    X_cwola_trainT21, Y_cwola_trainT21 = shuffle(X_cwola_trainT21, Y_cwola_trainT21, )

    sbmjjT21 = X_cwola_trainT21[:,0]
    sb_critT21 = ((sbmjjT21 > SR_low - SB_width)&(sbmjjT21 <= SR_high + SB_width))
    
    X_cwola_trainT21 = X_cwola_trainT21[sb_critT21]
    Y_cwola_trainT21 = Y_cwola_trainT21[sb_critT21]
    
    w_lowT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] <= SR_low).sum())
    w_highT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] >= SR_high).sum())

    W_cwola_trainT21 = np.ones_like(Y_cwola_trainT21)
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] <= SR_low)] = w_lowT21
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] >= SR_high)] = w_highT21
    
    X_cwola_valT21 = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
    Y_cwola_valT21 = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
    
    K.clear_session()
    model_cwolaT21_selu = Sequential()
    model_cwolaT21_selu.add(Dense(64, input_dim=N_inputs, activation='selu')) 
    model_cwolaT21_selu.add(Dense(64, activation='selu'))
    model_cwolaT21_selu.add(Dense(64, activation='selu'))
    model_cwolaT21_selu.add(Dense(1, activation='sigmoid'))
    model_cwolaT21_selu.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_cwolaT21_selu = model_cwolaT21_selu.fit(X_cwola_trainT21[:,1:], Y_cwola_trainT21, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0, sample_weight=W_cwola_trainT21)

    scores_cwolaT21_selu = model_cwolaT21_selu.predict(X_cwola_valT21[:,1:],batch_size=batch_size)
        
    fpr_cwolaT21_selu, tpr_cwolaT21_selu, _ = roc_curve(Y_cwola_valT21, scores_cwolaT21_selu)
    cwola_rocsT21_selu = cwola_rocsT21_selu.append({'fpr': fpr_cwolaT21_selu, 'tpr': tpr_cwolaT21_selu}, ignore_index=True)



## elu activation

In [ ]:
#CWoLa with T21 and elu 

cwola_rocsT21_elu = pd.DataFrame(columns=['fpr', 'tpr'])
SB_width = 0.3

for i in tqdm.tqdm(range(n_injectionsT21)):
    
    SR_signal_to_injectT21 = SR_signals_to_injectT21[i]
    SB_signal_to_injectT21 = SB_signals_to_injectT21[i]

    signal_to_testT21 = signals_to_testT21[i]
        
    X_cwola_trainT21 = np.concatenate([SR_signal_to_injectT21, SR_background_dataT21, SB_signal_to_injectT21, SB_background_dataT21])
    Y_cwola_trainT21 = np.concatenate([np.ones(len(SR_signal_to_injectT21)), np.ones(len(SR_background_dataT21)),
                                    np.zeros(len(SB_signal_to_injectT21)), np.zeros(len(SB_background_dataT21))])
    
    X_cwola_trainT21, Y_cwola_trainT21 = shuffle(X_cwola_trainT21, Y_cwola_trainT21, )

    sbmjjT21 = X_cwola_trainT21[:,0]
    sb_critT21 = ((sbmjjT21 > SR_low - SB_width)&(sbmjjT21 <= SR_high + SB_width))
    
    X_cwola_trainT21 = X_cwola_trainT21[sb_critT21]
    Y_cwola_trainT21 = Y_cwola_trainT21[sb_critT21]
    
    w_lowT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] <= SR_low).sum())
    w_highT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] >= SR_high).sum())

    W_cwola_trainT21 = np.ones_like(Y_cwola_trainT21)
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] <= SR_low)] = w_lowT21
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] >= SR_high)] = w_highT21
    
    X_cwola_valT21 = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
    Y_cwola_valT21 = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
    
    K.clear_session()
    model_cwolaT21_elu = Sequential()
    model_cwolaT21_elu.add(Dense(64, input_dim=N_inputs, activation='elu')) 
    model_cwolaT21_elu.add(Dense(64, activation='elu'))
    model_cwolaT21_elu.add(Dense(64, activation='elu'))
    model_cwolaT21_elu.add(Dense(1, activation='sigmoid'))
    model_cwolaT21_elu.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_cwolaT21_elu = model_cwolaT21_elu.fit(X_cwola_trainT21[:,1:], Y_cwola_trainT21, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0, sample_weight=W_cwola_trainT21)

    scores_cwolaT21_elu = model_cwolaT21_elu.predict(X_cwola_valT21[:,1:],batch_size=batch_size)
        
    fpr_cwolaT21_elu, tpr_cwolaT21_elu, _ = roc_curve(Y_cwola_valT21, scores_cwolaT21_elu)
    cwola_rocsT21_elu = cwola_rocsT21_elu.append({'fpr': fpr_cwolaT21_elu, 'tpr': tpr_cwolaT21_elu}, ignore_index=True)



## leaky relu

In [ ]:
#CWoLa with T21 and leaky relu

cwola_rocsT21_lrelu = pd.DataFrame(columns=['fpr', 'tpr'])
SB_width = 0.3

for i in tqdm.tqdm(range(n_injectionsT21)):
    
    SR_signal_to_injectT21 = SR_signals_to_injectT21[i]
    SB_signal_to_injectT21 = SB_signals_to_injectT21[i]

    signal_to_testT21 = signals_to_testT21[i]
        
    X_cwola_trainT21 = np.concatenate([SR_signal_to_injectT21, SR_background_dataT21, SB_signal_to_injectT21, SB_background_dataT21])
    Y_cwola_trainT21 = np.concatenate([np.ones(len(SR_signal_to_injectT21)), np.ones(len(SR_background_dataT21)),
                                    np.zeros(len(SB_signal_to_injectT21)), np.zeros(len(SB_background_dataT21))])
    
    X_cwola_trainT21, Y_cwola_trainT21 = shuffle(X_cwola_trainT21, Y_cwola_trainT21, )

    sbmjjT21 = X_cwola_trainT21[:,0]
    sb_critT21 = ((sbmjjT21 > SR_low - SB_width)&(sbmjjT21 <= SR_high + SB_width))
    
    X_cwola_trainT21 = X_cwola_trainT21[sb_critT21]
    Y_cwola_trainT21 = Y_cwola_trainT21[sb_critT21]
    
    w_lowT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] <= SR_low).sum())
    w_highT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] >= SR_high).sum())

    W_cwola_trainT21 = np.ones_like(Y_cwola_trainT21)
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] <= SR_low)] = w_lowT21
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] >= SR_high)] = w_highT21
    
    X_cwola_valT21 = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
    Y_cwola_valT21 = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
    
    K.clear_session()
    model_cwolaT21_lrelu = Sequential()
    model_cwolaT21_lrelu.add(Dense(64, input_dim=N_inputs, activation='leaky_relu')) 
    model_cwolaT21_lrelu.add(Dense(64, activation='leaky_relu'))
    model_cwolaT21_lrelu.add(Dense(64, activation='leaky_relu'))
    model_cwolaT21_lrelu.add(Dense(1, activation='sigmoid'))
    model_cwolaT21_lrelu.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_cwolaT21_lrelu = model_cwolaT21_lrelu.fit(X_cwola_trainT21[:,1:], Y_cwola_trainT21, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0, sample_weight=W_cwola_trainT21)

    scores_cwolaT21_lrelu = model_cwolaT21_lrelu.predict(X_cwola_valT21[:,1:],batch_size=batch_size)
        
    fpr_cwolaT21_lrelu, tpr_cwolaT21_lrelu, _ = roc_curve(Y_cwola_valT21, scores_cwolaT21_lrelu)
    cwola_rocsT21_lrelu = cwola_rocsT21_lrelu.append({'fpr': fpr_cwolaT21_lrelu, 'tpr': tpr_cwolaT21_lrelu}, ignore_index=True)

In [ ]:
with np.errstate(divide='ignore'):
    tpr_pts = np.linspace(0, 1, 10000)
    
    # CWoLa T21 selu
    fpr_interp = [None] * len(cwola_rocsT21_selu)

    for i, row in cwola_rocsT21_selu.iterrows():
        fpr, tpr = row
        interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

        fpr_pts = interp(tpr_pts)
        fpr_interp[i] = fpr_pts

    fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

    cwola_maxT21_selu = np.nanmax(fpr_interp, axis=0).data
    cwola_minT21_selu = np.nanmin(fpr_interp, axis=0).data
    cwola_medT21_selu = np.median(fpr_interp, axis=0).data
    cwola_meanT21_selu = np.nanmean(fpr_interp, axis=0).data
    cwola_stdT21_selu = np.nanstd(fpr_interp, axis=0).data
    
    sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
    cwola_smeanT21_selu = np.nanmean(sig_interp, axis=0).data
    cwola_sstdT21_selu = np.nanstd(sig_interp, axis=0).data
    
    
    # CWoLa T21 elu
    fpr_interp = [None] * len(cwola_rocsT21_elu)

    for i, row in cwola_rocsT21_elu.iterrows():
        fpr, tpr = row
        interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

        fpr_pts = interp(tpr_pts)
        fpr_interp[i] = fpr_pts

    fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

    cwola_maxT21_elu = np.nanmax(fpr_interp, axis=0).data
    cwola_minT21_elu = np.nanmin(fpr_interp, axis=0).data
    cwola_medT21_elu = np.median(fpr_interp, axis=0).data
    cwola_meanT21_elu = np.nanmean(fpr_interp, axis=0).data
    cwola_stdT21_elu = np.nanstd(fpr_interp, axis=0).data
    
    sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
    cwola_smeanT21_elu = np.nanmean(sig_interp, axis=0).data
    cwola_sstdT21_elu = np.nanstd(sig_interp, axis=0).data
    
    
    # CWoLa T21 leaky relu
    fpr_interp = [None] * len(cwola_rocsT21_lrelu)

    for i, row in cwola_rocsT21_lrelu.iterrows():
        fpr, tpr = row
        interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

        fpr_pts = interp(tpr_pts)
        fpr_interp[i] = fpr_pts

    fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

    cwola_maxT21_lrelu = np.nanmax(fpr_interp, axis=0).data
    cwola_minT21_lrelu = np.nanmin(fpr_interp, axis=0).data
    cwola_medT21_lrelu = np.median(fpr_interp, axis=0).data
    cwola_meanT21_lrelu = np.nanmean(fpr_interp, axis=0).data
    cwola_stdT21_lrelu = np.nanstd(fpr_interp, axis=0).data
    
    sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
    cwola_smeanT21_lrelu = np.nanmean(sig_interp, axis=0).data
    cwola_sstdT21_lrelu = np.nanstd(sig_interp, axis=0).data

In [ ]:
bands2 = {'cwola_T21relu': (cwola_maxT21, cwola_medT21, cwola_minT21, cwola_meanT21, cwola_stdT21, cwola_smeanT21, cwola_sstdT21, {'label':'ReLU', 'color':'blue'}),
          'cwola_T21selu': (cwola_maxT21_selu, cwola_medT21_selu, cwola_minT21_selu, cwola_meanT21_selu, cwola_stdT21_selu, cwola_smeanT21_selu, cwola_sstdT21_selu, {'label':'SELU', 'color':'red'}),
          'cwola_T21elu': (cwola_maxT21_elu, cwola_medT21_elu, cwola_minT21_elu, cwola_meanT21_elu, cwola_stdT21_elu, cwola_smeanT21_elu, cwola_sstdT21_elu, {'label':'ELU', 'color':'orange'}),
          'cwola_T21lrelu': (cwola_maxT21_lrelu, cwola_medT21_lrelu, cwola_minT21_lrelu, cwola_meanT21_lrelu, cwola_stdT21_lrelu, cwola_smeanT21_lrelu, cwola_sstdT21_lrelu, {'label':'Leaky ReLU', 'color':'green'}),
         }

In [ ]:
# Plot without banding
mylambda=.5
myklambda = 1 #np.round(len(SR_background_data)/len(SR_background_sim), 3)

fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1])
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.yscale("log")
plt.ylim([1,5*1e4])
ERR=False
#plt.plot(tpr_sup,1./fpr_sup,label="Supervised",color='green')

for band in bands2:
    if band in ['old-sacwola', 'hl-sacwola']:
        continue
    mu,std = bands2[band][3:5]

    plt.plot(tpr_pts, mu, **bands2[band][-1])
    if ERR:
        plt.fill_between(tpr_pts, mu - std, mu + std, alpha=.2, color=bands[band][-1]['color'])
#plt.plot(tpr_sup,1./tpr_sup,color="black",ls=":",label="Random")

plt.xlabel(r'Signal Efficiency (True Positive Rate)',fontsize=20)
plt.ylabel(r"Rejection (1/False Positive Rate)",fontsize=20)
plt.legend(frameon=False,fontsize=15,loc="upper right")

## Models for AUC

In [ ]:
#initial data

SR_low = 3.3
SR_high = 3.7
SB_low = 3.1
SB_high = 3.9
SB_width = 0.3
test_size = 0.5
EPOCHS=20
batch_size = 200

#mylambda=.5
myklambda = 1 #np.round(len(SR_background_data)/len(SR_background_sim), 3)

## 2 layers

In [ ]:
from tensorflow.keras.callbacks import Callback

class AUCRecordsL2(Callback, ):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"epoch {epoch}")
        
    def on_epoch_end(self, epoch, logs=None):
        global aucs_foravg_2lay, aucsSB_foravg_2lay, aucs_foravg_2lay, aucsSB_foravg_2lay, aucs_2lay, aucs_std_2lay, aucsSB_2lay, aucsSB_std_2lay, num_epochs
        preds_hold_2lay = self.model.predict(norm_func(X_test)[:,1:],batch_size=10000)[:,0]
        preds_holdSB_2lay = self.model.predict(norm_func(X_testSB)[:,1:],batch_size=10000)[:,0]
        num_epochs += [epoch]
        
        myauc_scan_2lay = roc_auc_score(Y_test, preds_hold_2lay)
        print(myauc_scan_2lay)
        myaucSB_scan_2lay = roc_auc_score(Y_testSB, preds_holdSB_2lay)
        print(myaucSB_scan_2lay)
        
        aucs_foravg_2lay += [myauc_scan_2lay]
        print(aucs_foravg_2lay)
        aucsSB_foravg_2lay += [myaucSB_scan_2lay]
        print(aucsSB_foravg_2lay)
        
        aucs_2lay+=[np.mean(aucs_foravg_2lay)]
        print(aucs_2lay)
        aucs_std_2lay += [np.std(aucs_foravg_2lay)]
        print(aucs_std_2lay)
        aucsSB_2lay+=[np.mean(aucsSB_foravg_2lay)]
        print(aucsSB_2lay)
        aucsSB_std_2lay += [np.std(aucsSB_foravg_2lay)]
        print(aucsSB_std_2lay)
        
        print(f"epoch {epoch}")

In [ ]:
    lambdas_2lay = []
    num_epochs = []
    aucs_2lay = []
    aucs_std_2lay = []
    aucsSB_2lay = []
    aucsSB_std_2lay = []
    aucs_foravg_2lay =[]
    aucsSB_foravg_2lay = []

    for k in range(1):
        model_cwola_scan_2lay = Sequential()
        model_cwola_scan_2lay.add(Dense(64, input_dim=4, activation='relu')) 
        model_cwola_scan_2lay.add(Dense(100, activation='relu'))
        model_cwola_scan_2lay.add(Dense(1, activation='sigmoid'))
        model_cwola_scan_2lay.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])
        
        # Generate training datasets
        SR_signal_to_injectT21 = SR_signals_to_injectT21[k]
        SB_signal_to_injectT21 = SB_signals_to_injectT21[k]

        signal_to_testT21 = signals_to_testT21[k]

        X_kcwola_train_2lay = np.concatenate([
            SR_signal_to_injectT21, SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] < SR_low], 
            SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] > SR_high],
            SR_background_dataT21, SB_background_dataT21[SB_background_dataT21[:,0] < SR_low], 
            SB_background_dataT21[SB_background_dataT21[:,0] > SR_high],
            SR_background_simT21, SB_background_simT21[SB_background_simT21[:,0] < SR_low], 
            SB_background_simT21[SB_background_simT21[:,0] > SR_high]
        ])
        Y_kcwola_train_2lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), np.zeros(len(SB_signal_to_injectT21)),
            np.ones(len(SR_background_dataT21)), np.zeros(len(SB_background_dataT21)),
            np.zeros(len(SR_background_simT21)), np.ones(len(SB_background_simT21))
        ])
        
        n_low_data = ((SB_background_dataT21[:,0] > (SB_low)) & ((SB_background_dataT21[:,0] <= SR_low))).sum()
        n_low_sim = ((SB_background_simT21[:,0] > (SB_low)) & ((SB_background_simT21[:,0] <= SR_low))).sum()
        n_low_sig = ((SB_signal_to_injectT21[:,0] > (SB_low)) & ((SB_signal_to_injectT21[:,0] <= SR_low))).sum()

        n_hi_data = ((SB_background_dataT21[:,0] < (SB_high)) & ((SB_background_dataT21[:,0] >= SR_high))).sum()
        n_hi_sim = ((SB_background_simT21[:,0] < (SB_high)) & ((SB_background_simT21[:,0] >= SR_high))).sum()
        n_hi_sig = ((SB_signal_to_injectT21[:,0] < (SB_high)) & ((SB_signal_to_injectT21[:,0] >= SR_high))).sum()

        n_sr_data = len(SR_background_dataT21)
        n_sr_sim = len(SR_background_simT21)
        n_sr_sig = len(SR_signal_to_injectT21)

        w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
        w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)

        w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
        w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
        
        W_kcwola_train_2lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), w_low*np.ones((SB_signal_to_injectT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_signal_to_injectT21[:,0] > SR_high).sum()),
            np.ones(len(SR_background_dataT21)), w_low*np.ones((SB_background_dataT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_background_dataT21[:,0] > SR_high).sum()),
            myklambda*np.ones(len(SR_background_simT21)), 
            w_low_sim*myklambda*np.ones((SB_background_simT21[:,0] <= SR_low).sum()), 
            w_high_sim*myklambda*np.ones((SB_background_simT21[:,0] > SR_high).sum())
        ])
    
        sbmjj = X_kcwola_train_2lay[:,0]
        sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

        X_kcwola_train_2lay = X_kcwola_train_2lay[sb_crit]
        Y_kcwola_train_2lay = Y_kcwola_train_2lay[sb_crit]
        W_kcwola_train_2lay = W_kcwola_train_2lay[sb_crit]


        X_kcwola_train_2lay, Y_kcwola_train_2lay, W_kcwola_train_2lay = shuffle(X_kcwola_train_2lay, Y_kcwola_train_2lay, 
                                                                       W_kcwola_train_2lay)

    
        # Generate testing datasets
        X_test = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
        Y_test = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
        
        X_testSB = np.concatenate([SR_background_data_fortestT21,SB_background_data_fortestT21])
        Y_testSB = np.concatenate([np.ones(len(SR_background_data_fortestT21)),
                                   np.zeros(len(SB_background_data_fortestT21))])

        model_cwola_scan_2lay.fit(norm_func(X_kcwola_train_2lay)[:,1:],Y_kcwola_train_2lay, epochs=EPOCHS, 
                               batch_size=batch_size,validation_data=(norm_func(X_test)[:,1:],Y_test),
                               sample_weight=W_kcwola_train_2lay, verbose=2, callbacks=[AUCRecordsL2()])

        pass
        

    
    

In [ ]:
aucs_2lay = np.asarray(aucs_2lay)
aucs_std_2lay = np.asarray(aucs_std_2lay)

aucsSB_2lay = np.asarray(aucsSB_2lay)
aucsSB_std_2lay = np.asarray(aucsSB_std_2lay)

print(aucs_2lay)
print(aucsSB_2lay)

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_2lay,label="Signal versus Background",linewidth=3)
#plt.fill_between(epochs,aucs+aucs_std, aucs-aucs_std, alpha=0.2)
plt.plot(num_epochs, aucsSB_2lay,label="Background in SR versus SB",ls="--",linewidth=3)
#plt.fill_between(epochs,aucsSB+aucsSB_std, aucsSB-aucsSB_std,alpha=0.2)
plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20)
plt.tight_layout()
plt.ylim([0,1])

## 3 layers

In [ ]:
class AUCRecordsL3(Callback, ):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"epoch {epoch}")
        
    def on_epoch_end(self, epoch, logs=None):
        global aucs_foravg_3lay, aucsSB_foravg_3lay, aucs_foravg_3lay, aucsSB_foravg_3lay, aucs_3lay, aucs_std_3lay, aucsSB_3lay, aucsSB_std_3lay, num_epochs
        preds_hold_3lay = self.model.predict(norm_func(X_test)[:,1:],batch_size=10000)[:,0]
        preds_holdSB_3lay = self.model.predict(norm_func(X_testSB)[:,1:],batch_size=10000)[:,0]
        num_epochs += [epoch]
        
        myauc_scan_3lay = roc_auc_score(Y_test, preds_hold_3lay)
        print(myauc_scan_3lay)
        myaucSB_scan_3lay = roc_auc_score(Y_testSB, preds_holdSB_3lay)
        print(myaucSB_scan_3lay)
        
        aucs_foravg_3lay += [myauc_scan_3lay]
        print(aucs_foravg_3lay)
        aucsSB_foravg_3lay += [myaucSB_scan_3lay]
        print(aucsSB_foravg_3lay)
        
        aucs_3lay+=[np.mean(aucs_foravg_3lay)]
        print(aucs_3lay)
        aucs_std_3lay += [np.std(aucs_foravg_3lay)]
        print(aucs_std_3lay)
        aucsSB_3lay+=[np.mean(aucsSB_foravg_3lay)]
        print(aucsSB_3lay)
        aucsSB_std_3lay += [np.std(aucsSB_foravg_3lay)]
        print(aucsSB_std_3lay)
        
        print(f"epoch {epoch}")

In [ ]:
    lambdas_3lay = []
    num_epochs = []
    aucs_3lay = []
    aucs_std_3lay = []
    aucsSB_3lay = []
    aucsSB_std_3lay = []
    aucs_foravg_3lay =[]
    aucsSB_foravg_3lay = []

    for k in range(1):
        model_cwola_scan_3lay = Sequential()
        model_cwola_scan_3lay.add(Dense(64, input_dim=4, activation='relu')) 
        model_cwola_scan_3lay.add(Dense(100, activation='relu'))
        model_cwola_scan_3lay.add(Dense(64, activation='relu'))
        model_cwola_scan_3lay.add(Dense(1, activation='sigmoid'))
        model_cwola_scan_3lay.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])
        
        # Generate training datasets
        SR_signal_to_injectT21 = SR_signals_to_injectT21[k]
        SB_signal_to_injectT21 = SB_signals_to_injectT21[k]

        signal_to_testT21 = signals_to_testT21[k]

        X_kcwola_train_3lay = np.concatenate([
            SR_signal_to_injectT21, SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] < SR_low], 
            SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] > SR_high],
            SR_background_dataT21, SB_background_dataT21[SB_background_dataT21[:,0] < SR_low], 
            SB_background_dataT21[SB_background_dataT21[:,0] > SR_high],
            SR_background_simT21, SB_background_simT21[SB_background_simT21[:,0] < SR_low], 
            SB_background_simT21[SB_background_simT21[:,0] > SR_high]
        ])
        Y_kcwola_train_3lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), np.zeros(len(SB_signal_to_injectT21)),
            np.ones(len(SR_background_dataT21)), np.zeros(len(SB_background_dataT21)),
            np.zeros(len(SR_background_simT21)), np.ones(len(SB_background_simT21))
        ])
        
        n_low_data = ((SB_background_dataT21[:,0] > (SB_low)) & ((SB_background_dataT21[:,0] <= SR_low))).sum()
        n_low_sim = ((SB_background_simT21[:,0] > (SB_low)) & ((SB_background_simT21[:,0] <= SR_low))).sum()
        n_low_sig = ((SB_signal_to_injectT21[:,0] > (SB_low)) & ((SB_signal_to_injectT21[:,0] <= SR_low))).sum()

        n_hi_data = ((SB_background_dataT21[:,0] < (SB_high)) & ((SB_background_dataT21[:,0] >= SR_high))).sum()
        n_hi_sim = ((SB_background_simT21[:,0] < (SB_high)) & ((SB_background_simT21[:,0] >= SR_high))).sum()
        n_hi_sig = ((SB_signal_to_injectT21[:,0] < (SB_high)) & ((SB_signal_to_injectT21[:,0] >= SR_high))).sum()

        n_sr_data = len(SR_background_dataT21)
        n_sr_sim = len(SR_background_simT21)
        n_sr_sig = len(SR_signal_to_injectT21)

        w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
        w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)

        w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
        w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
        
        W_kcwola_train_3lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), w_low*np.ones((SB_signal_to_injectT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_signal_to_injectT21[:,0] > SR_high).sum()),
            np.ones(len(SR_background_dataT21)), w_low*np.ones((SB_background_dataT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_background_dataT21[:,0] > SR_high).sum()),
            myklambda*np.ones(len(SR_background_simT21)), 
            w_low_sim*myklambda*np.ones((SB_background_simT21[:,0] <= SR_low).sum()), 
            w_high_sim*myklambda*np.ones((SB_background_simT21[:,0] > SR_high).sum())
        ])
    
        sbmjj = X_kcwola_train_3lay[:,0]
        sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

        X_kcwola_train_3lay = X_kcwola_train_3lay[sb_crit]
        Y_kcwola_train_3lay = Y_kcwola_train_3lay[sb_crit]
        W_kcwola_train_3lay = W_kcwola_train_3lay[sb_crit]


        X_kcwola_train_3lay, Y_kcwola_train_3lay, W_kcwola_train_3lay = shuffle(X_kcwola_train_3lay, Y_kcwola_train_3lay, 
                                                                       W_kcwola_train_3lay)

    
        # Generate testing datasets
        X_test = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
        Y_test = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
        
        X_testSB = np.concatenate([SR_background_data_fortestT21,SB_background_data_fortestT21])
        Y_testSB = np.concatenate([np.ones(len(SR_background_data_fortestT21)),
                                   np.zeros(len(SB_background_data_fortestT21))])

        model_cwola_scan_3lay.fit(norm_func(X_kcwola_train_3lay)[:,1:],Y_kcwola_train_3lay, epochs=EPOCHS, 
                               batch_size=batch_size,validation_data=(norm_func(X_test)[:,1:],Y_test),
                               sample_weight=W_kcwola_train_3lay, verbose=2, callbacks=[AUCRecordsL3()])

        pass


In [ ]:
aucs_3lay = np.asarray(aucs_3lay)
aucs_std_3lay = np.asarray(aucs_std_3lay)

aucsSB_3lay = np.asarray(aucsSB_3lay)
aucsSB_std_3lay = np.asarray(aucsSB_std_3lay)

print(aucs_3lay)
print(aucsSB_3lay)

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_3lay,label="Signal versus Background",linewidth=3)
#plt.fill_between(epochs,aucs+aucs_std, aucs-aucs_std, alpha=0.2)
plt.plot(num_epochs, aucsSB_3lay,label="Background in SR versus SB",ls="--",linewidth=3)
#plt.fill_between(epochs,aucsSB+aucsSB_std, aucsSB-aucsSB_std,alpha=0.2)
plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20)
plt.tight_layout()
plt.ylim([0,1])

## 4 layers

In [ ]:
class AUCRecordsL4(Callback, ):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"epoch {epoch}")
        
    def on_epoch_end(self, epoch, logs=None):
        global aucs_foravg_4lay, aucsSB_foravg_4lay, aucs_foravg_4lay, aucsSB_foravg_4lay, aucs_4lay, aucs_std_4lay, aucsSB_4lay, aucsSB_std_4lay, num_epochs
        preds_hold_4lay = self.model.predict(norm_func(X_test)[:,1:],batch_size=10000)[:,0]
        preds_holdSB_4lay = self.model.predict(norm_func(X_testSB)[:,1:],batch_size=10000)[:,0]
        num_epochs += [epoch]
        
        myauc_scan_4lay = roc_auc_score(Y_test, preds_hold_4lay)
        print(myauc_scan_4lay)
        myaucSB_scan_4lay = roc_auc_score(Y_testSB, preds_holdSB_4lay)
        print(myaucSB_scan_4lay)
        
        aucs_foravg_4lay += [myauc_scan_4lay]
        print(aucs_foravg_4lay)
        aucsSB_foravg_4lay += [myaucSB_scan_4lay]
        print(aucsSB_foravg_4lay)
        
        aucs_4lay+=[np.mean(aucs_foravg_4lay)]
        print(aucs_4lay)
        aucs_std_4lay += [np.std(aucs_foravg_4lay)]
        print(aucs_std_4lay)
        aucsSB_4lay+=[np.mean(aucsSB_foravg_4lay)]
        print(aucsSB_4lay)
        aucsSB_std_4lay += [np.std(aucsSB_foravg_4lay)]
        print(aucsSB_std_4lay)
        
        print(f"epoch {epoch}")

In [ ]:
    lambdas_4lay = []
    num_epochs = []
    aucs_4lay = []
    aucs_std_4lay = []
    aucsSB_4lay = []
    aucsSB_std_4lay = []
    aucs_foravg_4lay =[]
    aucsSB_foravg_4lay = []

    for k in range(1):
        model_cwola_scan_4lay = Sequential()
        model_cwola_scan_4lay.add(Dense(64, input_dim=4, activation='relu')) 
        model_cwola_scan_4lay.add(Dense(100, activation='relu'))
        model_cwola_scan_4lay.add(Dense(64, activation='relu'))
        model_cwola_scan_4lay.add(Dense(100, activation='relu'))
        model_cwola_scan_4lay.add(Dense(1, activation='sigmoid'))
        model_cwola_scan_4lay.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])
        
        # Generate training datasets
        SR_signal_to_injectT21 = SR_signals_to_injectT21[k]
        SB_signal_to_injectT21 = SB_signals_to_injectT21[k]

        signal_to_testT21 = signals_to_testT21[k]

        X_kcwola_train_4lay = np.concatenate([
            SR_signal_to_injectT21, SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] < SR_low], 
            SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] > SR_high],
            SR_background_dataT21, SB_background_dataT21[SB_background_dataT21[:,0] < SR_low], 
            SB_background_dataT21[SB_background_dataT21[:,0] > SR_high],
            SR_background_simT21, SB_background_simT21[SB_background_simT21[:,0] < SR_low], 
            SB_background_simT21[SB_background_simT21[:,0] > SR_high]
        ])
        Y_kcwola_train_4lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), np.zeros(len(SB_signal_to_injectT21)),
            np.ones(len(SR_background_dataT21)), np.zeros(len(SB_background_dataT21)),
            np.zeros(len(SR_background_simT21)), np.ones(len(SB_background_simT21))
        ])
        
        n_low_data = ((SB_background_dataT21[:,0] > (SB_low)) & ((SB_background_dataT21[:,0] <= SR_low))).sum()
        n_low_sim = ((SB_background_simT21[:,0] > (SB_low)) & ((SB_background_simT21[:,0] <= SR_low))).sum()
        n_low_sig = ((SB_signal_to_injectT21[:,0] > (SB_low)) & ((SB_signal_to_injectT21[:,0] <= SR_low))).sum()

        n_hi_data = ((SB_background_dataT21[:,0] < (SB_high)) & ((SB_background_dataT21[:,0] >= SR_high))).sum()
        n_hi_sim = ((SB_background_simT21[:,0] < (SB_high)) & ((SB_background_simT21[:,0] >= SR_high))).sum()
        n_hi_sig = ((SB_signal_to_injectT21[:,0] < (SB_high)) & ((SB_signal_to_injectT21[:,0] >= SR_high))).sum()

        n_sr_data = len(SR_background_dataT21)
        n_sr_sim = len(SR_background_simT21)
        n_sr_sig = len(SR_signal_to_injectT21)

        w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
        w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)

        w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
        w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
        
        W_kcwola_train_4lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), w_low*np.ones((SB_signal_to_injectT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_signal_to_injectT21[:,0] > SR_high).sum()),
            np.ones(len(SR_background_dataT21)), w_low*np.ones((SB_background_dataT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_background_dataT21[:,0] > SR_high).sum()),
            myklambda*np.ones(len(SR_background_simT21)), 
            w_low_sim*myklambda*np.ones((SB_background_simT21[:,0] <= SR_low).sum()), 
            w_high_sim*myklambda*np.ones((SB_background_simT21[:,0] > SR_high).sum())
        ])
    
        sbmjj = X_kcwola_train_4lay[:,0]
        sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

        X_kcwola_train_4lay = X_kcwola_train_4lay[sb_crit]
        Y_kcwola_train_4lay = Y_kcwola_train_4lay[sb_crit]
        W_kcwola_train_4lay = W_kcwola_train_4lay[sb_crit]


        X_kcwola_train_4lay, Y_kcwola_train_4lay, W_kcwola_train_4lay = shuffle(X_kcwola_train_4lay, Y_kcwola_train_4lay, 
                                                                       W_kcwola_train_4lay)

    
        # Generate testing datasets
        X_test = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
        Y_test = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
        
        X_testSB = np.concatenate([SR_background_data_fortestT21,SB_background_data_fortestT21])
        Y_testSB = np.concatenate([np.ones(len(SR_background_data_fortestT21)),
                                   np.zeros(len(SB_background_data_fortestT21))])

        model_cwola_scan_4lay.fit(norm_func(X_kcwola_train_4lay)[:,1:],Y_kcwola_train_4lay, epochs=EPOCHS, 
                               batch_size=batch_size,validation_data=(norm_func(X_test)[:,1:],Y_test),
                               sample_weight=W_kcwola_train_4lay, verbose=2, callbacks=[AUCRecordsL4()])

        pass


In [ ]:
aucs_4lay = np.asarray(aucs_4lay)
aucs_std_4lay = np.asarray(aucs_std_4lay)

aucsSB_4lay = np.asarray(aucsSB_4lay)
aucsSB_std_4lay = np.asarray(aucsSB_std_4lay)

print(aucs_4lay)
print(aucsSB_4lay)

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_4lay,label="Signal versus Background",linewidth=3)
#plt.fill_between(epochs,aucs+aucs_std, aucs-aucs_std, alpha=0.2)
plt.plot(num_epochs, aucsSB_4lay,label="Background in SR versus SB",ls="--",linewidth=3)
#plt.fill_between(epochs,aucsSB+aucsSB_std, aucsSB-aucsSB_std,alpha=0.2)
plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20)
plt.tight_layout()
plt.ylim([0,1])

## 5 layers

In [ ]:
class AUCRecordsL5(Callback, ):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"epoch {epoch}")
        
    def on_epoch_end(self, epoch, logs=None):
        global aucs_foravg_5lay, aucsSB_foravg_5lay, aucs_foravg_5lay, aucsSB_foravg_5lay, aucs_5lay, aucs_std_5lay, aucsSB_5lay, aucsSB_std_5lay, num_epochs
        preds_hold_5lay = self.model.predict(norm_func(X_test)[:,1:],batch_size=10000)[:,0]
        preds_holdSB_5lay = self.model.predict(norm_func(X_testSB)[:,1:],batch_size=10000)[:,0]
        num_epochs += [epoch]
        
        myauc_scan_5lay = roc_auc_score(Y_test, preds_hold_5lay)
        print(myauc_scan_5lay)
        myaucSB_scan_5lay = roc_auc_score(Y_testSB, preds_holdSB_5lay)
        print(myaucSB_scan_5lay)
        
        aucs_foravg_5lay += [myauc_scan_5lay]
        print(aucs_foravg_5lay)
        aucsSB_foravg_5lay += [myaucSB_scan_5lay]
        print(aucsSB_foravg_5lay)
        
        aucs_5lay+=[np.mean(aucs_foravg_5lay)]
        print(aucs_5lay)
        aucs_std_5lay += [np.std(aucs_foravg_5lay)]
        print(aucs_std_5lay)
        aucsSB_5lay+=[np.mean(aucsSB_foravg_5lay)]
        print(aucsSB_5lay)
        aucsSB_std_5lay += [np.std(aucsSB_foravg_5lay)]
        print(aucsSB_std_5lay)
        
        print(f"epoch {epoch}")

In [ ]:
    lambdas_5lay = []
    num_epochs = []
    aucs_5lay = []
    aucs_std_5lay = []
    aucsSB_5lay = []
    aucsSB_std_5lay = []
    aucs_foravg_5lay =[]
    aucsSB_foravg_5lay = []

    for k in range(1):
        model_cwola_scan_5lay = Sequential()
        model_cwola_scan_5lay.add(Dense(64, input_dim=4, activation='relu')) 
        model_cwola_scan_5lay.add(Dense(100, activation='relu'))
        model_cwola_scan_5lay.add(Dense(64, activation='relu'))
        model_cwola_scan_5lay.add(Dense(100, activation='relu'))
        model_cwola_scan_5lay.add(Dense(64, activation='relu'))
        model_cwola_scan_5lay.add(Dense(1, activation='sigmoid'))
        model_cwola_scan_5lay.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])
        
        # Generate training datasets
        SR_signal_to_injectT21 = SR_signals_to_injectT21[k]
        SB_signal_to_injectT21 = SB_signals_to_injectT21[k]

        signal_to_testT21 = signals_to_testT21[k]

        X_kcwola_train_5lay = np.concatenate([
            SR_signal_to_injectT21, SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] < SR_low], 
            SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] > SR_high],
            SR_background_dataT21, SB_background_dataT21[SB_background_dataT21[:,0] < SR_low], 
            SB_background_dataT21[SB_background_dataT21[:,0] > SR_high],
            SR_background_simT21, SB_background_simT21[SB_background_simT21[:,0] < SR_low], 
            SB_background_simT21[SB_background_simT21[:,0] > SR_high]
        ])
        Y_kcwola_train_5lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), np.zeros(len(SB_signal_to_injectT21)),
            np.ones(len(SR_background_dataT21)), np.zeros(len(SB_background_dataT21)),
            np.zeros(len(SR_background_simT21)), np.ones(len(SB_background_simT21))
        ])
        
        n_low_data = ((SB_background_dataT21[:,0] > (SB_low)) & ((SB_background_dataT21[:,0] <= SR_low))).sum()
        n_low_sim = ((SB_background_simT21[:,0] > (SB_low)) & ((SB_background_simT21[:,0] <= SR_low))).sum()
        n_low_sig = ((SB_signal_to_injectT21[:,0] > (SB_low)) & ((SB_signal_to_injectT21[:,0] <= SR_low))).sum()

        n_hi_data = ((SB_background_dataT21[:,0] < (SB_high)) & ((SB_background_dataT21[:,0] >= SR_high))).sum()
        n_hi_sim = ((SB_background_simT21[:,0] < (SB_high)) & ((SB_background_simT21[:,0] >= SR_high))).sum()
        n_hi_sig = ((SB_signal_to_injectT21[:,0] < (SB_high)) & ((SB_signal_to_injectT21[:,0] >= SR_high))).sum()

        n_sr_data = len(SR_background_dataT21)
        n_sr_sim = len(SR_background_simT21)
        n_sr_sig = len(SR_signal_to_injectT21)

        w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
        w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)

        w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
        w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
        
        W_kcwola_train_5lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), w_low*np.ones((SB_signal_to_injectT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_signal_to_injectT21[:,0] > SR_high).sum()),
            np.ones(len(SR_background_dataT21)), w_low*np.ones((SB_background_dataT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_background_dataT21[:,0] > SR_high).sum()),
            myklambda*np.ones(len(SR_background_simT21)), 
            w_low_sim*myklambda*np.ones((SB_background_simT21[:,0] <= SR_low).sum()), 
            w_high_sim*myklambda*np.ones((SB_background_simT21[:,0] > SR_high).sum())
        ])
    
        sbmjj = X_kcwola_train_5lay[:,0]
        sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

        X_kcwola_train_5lay = X_kcwola_train_5lay[sb_crit]
        Y_kcwola_train_5lay = Y_kcwola_train_5lay[sb_crit]
        W_kcwola_train_5lay = W_kcwola_train_5lay[sb_crit]


        X_kcwola_train_5lay, Y_kcwola_train_5lay, W_kcwola_train_5lay = shuffle(X_kcwola_train_5lay, Y_kcwola_train_5lay, 
                                                                       W_kcwola_train_5lay)

    
        # Generate testing datasets
        X_test = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
        Y_test = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
        
        X_testSB = np.concatenate([SR_background_data_fortestT21,SB_background_data_fortestT21])
        Y_testSB = np.concatenate([np.ones(len(SR_background_data_fortestT21)),
                                   np.zeros(len(SB_background_data_fortestT21))])

        model_cwola_scan_5lay.fit(norm_func(X_kcwola_train_5lay)[:,1:],Y_kcwola_train_5lay, epochs=EPOCHS, 
                               batch_size=batch_size,validation_data=(norm_func(X_test)[:,1:],Y_test),
                               sample_weight=W_kcwola_train_5lay, verbose=2, callbacks=[AUCRecordsL5()])

        pass


In [ ]:
aucs_5lay = np.asarray(aucs_5lay)
aucs_std_5lay = np.asarray(aucs_std_5lay)

aucsSB_5lay = np.asarray(aucsSB_5lay)
aucsSB_std_5lay = np.asarray(aucsSB_std_5lay)

print(aucs_5lay)
print(aucsSB_5lay)

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_5lay,label="Signal versus Background",linewidth=3)
#plt.fill_between(epochs,aucs+aucs_std, aucs-aucs_std, alpha=0.2)
plt.plot(num_epochs, aucsSB_5lay,label="Background in SR versus SB",ls="--",linewidth=3)
#plt.fill_between(epochs,aucsSB+aucsSB_std, aucsSB-aucsSB_std,alpha=0.2)
plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20)
plt.tight_layout()
plt.ylim([0,1])

# AUC all architectures

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_2lay,label="Signal versus Background Model 2 layers architecture",linewidth=3,color='red')
plt.plot(num_epochs, aucsSB_2lay,label="Background in SR versus SB 2 layers architecture",ls="--",linewidth=3,color='red')

plt.plot(num_epochs, aucs_3lay,label="Signal versus Background 3 layers architecture",linewidth=3,color='blue')
plt.plot(num_epochs, aucsSB_3lay,label="Background in SR versus SB 3 layers architecture",ls="--",linewidth=3,color='blue')

plt.plot(num_epochs, aucs_4lay,label="Signal versus Background 4 layers architecture",linewidth=3,color='green')
plt.plot(num_epochs, aucsSB_4lay,label="Background in SR versus SB 4 layers architecture",ls="--",linewidth=3,color='green')

plt.plot(num_epochs, aucs_5lay,label="Signal versus Background 5 layers architecture",linewidth=3,color='orange')
plt.plot(num_epochs, aucsSB_5lay,label="Background in SR versus SB 5 layers architecture",ls="--",linewidth=3,color='orange')

plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20, bbox_to_anchor=(1.05, 1.0))
plt.tight_layout()
plt.ylim([0,1])

In [ ]:
class AUCRecordsL15(Callback, ):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"epoch {epoch}")
        
    def on_epoch_end(self, epoch, logs=None):
        global aucs_foravg_15lay, aucsSB_foravg_15lay, aucs_foravg_15lay, aucsSB_foravg_15lay, aucs_15lay, aucs_std_15lay, aucsSB_15lay, aucsSB_std_15lay, num_epochs
        preds_hold_15lay = self.model.predict(norm_func(X_test)[:,1:],batch_size=10000)[:,0]
        preds_holdSB_15lay = self.model.predict(norm_func(X_testSB)[:,1:],batch_size=10000)[:,0]
        num_epochs += [epoch]
        
        myauc_scan_15lay = roc_auc_score(Y_test, preds_hold_15lay)
        print(myauc_scan_15lay)
        myaucSB_scan_15lay = roc_auc_score(Y_testSB, preds_holdSB_15lay)
        print(myaucSB_scan_15lay)
        
        aucs_foravg_15lay += [myauc_scan_15lay]
        print(aucs_foravg_15lay)
        aucsSB_foravg_15lay += [myaucSB_scan_15lay]
        print(aucsSB_foravg_15lay)
        
        aucs_15lay+=[np.mean(aucs_foravg_15lay)]
        print(aucs_15lay)
        aucs_std_15lay += [np.std(aucs_foravg_15lay)]
        print(aucs_std_15lay)
        aucsSB_15lay+=[np.mean(aucsSB_foravg_15lay)]
        print(aucsSB_15lay)
        aucsSB_std_15lay += [np.std(aucsSB_foravg_15lay)]
        print(aucsSB_std_15lay)
        
        print(f"epoch {epoch}")

In [ ]:
    lambdas_15lay = []
    num_epochs = []
    aucs_15lay = []
    aucs_std_15lay = []
    aucsSB_15lay = []
    aucsSB_std_15lay = []
    aucs_foravg_15lay =[]
    aucsSB_foravg_15lay = []

    for k in range(1):
        model_cwola_scan_15lay = Sequential()
        model_cwola_scan_15lay.add(Dense(64, input_dim=4, activation='relu')) 
        model_cwola_scan_15lay.add(Dense(100, activation='relu'))
        model_cwola_scan_15lay.add(Dense(64, activation='relu'))
        model_cwola_scan_15lay.add(Dense(100, activation='relu'))
        model_cwola_scan_15lay.add(Dense(64, activation='relu'))
        model_cwola_scan_15lay.add(Dense(100, activation='relu'))
        model_cwola_scan_15lay.add(Dense(64, activation='relu'))
        model_cwola_scan_15lay.add(Dense(100, activation='relu'))
        model_cwola_scan_15lay.add(Dense(64, activation='relu'))
        model_cwola_scan_15lay.add(Dense(100, activation='relu'))
        model_cwola_scan_15lay.add(Dense(64, activation='relu'))
        model_cwola_scan_15lay.add(Dense(100, activation='relu'))
        model_cwola_scan_15lay.add(Dense(64, activation='relu'))
        model_cwola_scan_15lay.add(Dense(100, activation='relu'))
        model_cwola_scan_15lay.add(Dense(64, activation='relu'))
        model_cwola_scan_15lay.add(Dense(1, activation='sigmoid'))
        model_cwola_scan_15lay.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])
        
        # Generate training datasets
        SR_signal_to_injectT21 = SR_signals_to_injectT21[k]
        SB_signal_to_injectT21 = SB_signals_to_injectT21[k]

        signal_to_testT21 = signals_to_testT21[k]

        X_kcwola_train_15lay = np.concatenate([
            SR_signal_to_injectT21, SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] < SR_low], 
            SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] > SR_high],
            SR_background_dataT21, SB_background_dataT21[SB_background_dataT21[:,0] < SR_low], 
            SB_background_dataT21[SB_background_dataT21[:,0] > SR_high],
            SR_background_simT21, SB_background_simT21[SB_background_simT21[:,0] < SR_low], 
            SB_background_simT21[SB_background_simT21[:,0] > SR_high]
        ])
        Y_kcwola_train_15lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), np.zeros(len(SB_signal_to_injectT21)),
            np.ones(len(SR_background_dataT21)), np.zeros(len(SB_background_dataT21)),
            np.zeros(len(SR_background_simT21)), np.ones(len(SB_background_simT21))
        ])
        
        n_low_data = ((SB_background_dataT21[:,0] > (SB_low)) & ((SB_background_dataT21[:,0] <= SR_low))).sum()
        n_low_sim = ((SB_background_simT21[:,0] > (SB_low)) & ((SB_background_simT21[:,0] <= SR_low))).sum()
        n_low_sig = ((SB_signal_to_injectT21[:,0] > (SB_low)) & ((SB_signal_to_injectT21[:,0] <= SR_low))).sum()

        n_hi_data = ((SB_background_dataT21[:,0] < (SB_high)) & ((SB_background_dataT21[:,0] >= SR_high))).sum()
        n_hi_sim = ((SB_background_simT21[:,0] < (SB_high)) & ((SB_background_simT21[:,0] >= SR_high))).sum()
        n_hi_sig = ((SB_signal_to_injectT21[:,0] < (SB_high)) & ((SB_signal_to_injectT21[:,0] >= SR_high))).sum()

        n_sr_data = len(SR_background_dataT21)
        n_sr_sim = len(SR_background_simT21)
        n_sr_sig = len(SR_signal_to_injectT21)

        w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
        w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)

        w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
        w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
        
        W_kcwola_train_15lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), w_low*np.ones((SB_signal_to_injectT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_signal_to_injectT21[:,0] > SR_high).sum()),
            np.ones(len(SR_background_dataT21)), w_low*np.ones((SB_background_dataT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_background_dataT21[:,0] > SR_high).sum()),
            myklambda*np.ones(len(SR_background_simT21)), 
            w_low_sim*myklambda*np.ones((SB_background_simT21[:,0] <= SR_low).sum()), 
            w_high_sim*myklambda*np.ones((SB_background_simT21[:,0] > SR_high).sum())
        ])
    
        sbmjj = X_kcwola_train_15lay[:,0]
        sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

        X_kcwola_train_15lay = X_kcwola_train_15lay[sb_crit]
        Y_kcwola_train_15lay = Y_kcwola_train_15lay[sb_crit]
        W_kcwola_train_15lay = W_kcwola_train_15lay[sb_crit]


        X_kcwola_train_15lay, Y_kcwola_train_15lay, W_kcwola_train_15lay = shuffle(X_kcwola_train_15lay, Y_kcwola_train_15lay, 
                                                                       W_kcwola_train_15lay)

    
        # Generate testing datasets
        X_test = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
        Y_test = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
        
        X_testSB = np.concatenate([SR_background_data_fortestT21,SB_background_data_fortestT21])
        Y_testSB = np.concatenate([np.ones(len(SR_background_data_fortestT21)),
                                   np.zeros(len(SB_background_data_fortestT21))])

        model_cwola_scan_15lay.fit(norm_func(X_kcwola_train_15lay)[:,1:],Y_kcwola_train_15lay, epochs=EPOCHS, 
                               batch_size=batch_size,validation_data=(norm_func(X_test)[:,1:],Y_test),
                               sample_weight=W_kcwola_train_15lay, verbose=2, callbacks=[AUCRecordsL15()])

        pass


In [ ]:
aucs_15lay = np.asarray(aucs_15lay)
aucs_std_15lay = np.asarray(aucs_std_15lay)

aucsSB_15lay = np.asarray(aucsSB_15lay)
aucsSB_std_15lay = np.asarray(aucsSB_std_15lay)

print(aucs_15lay)
print(aucsSB_15lay)

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_2lay,label="Signal versus Background Model 2 layers architecture",linewidth=3,color='red')
plt.plot(num_epochs, aucsSB_2lay,label="Background in SR versus SB 2 layers architecture",ls="--",linewidth=3,color='red')

plt.plot(num_epochs, aucs_3lay,label="Signal versus Background 3 layers architecture",linewidth=3,color='blue')
plt.plot(num_epochs, aucsSB_3lay,label="Background in SR versus SB 3 layers architecture",ls="--",linewidth=3,color='blue')

plt.plot(num_epochs, aucs_4lay,label="Signal versus Background 4 layers architecture",linewidth=3,color='green')
plt.plot(num_epochs, aucsSB_4lay,label="Background in SR versus SB 4 layers architecture",ls="--",linewidth=3,color='green')

plt.plot(num_epochs, aucs_5lay,label="Signal versus Background 5 layers architecture",linewidth=3,color='orange')
plt.plot(num_epochs, aucsSB_5lay,label="Background in SR versus SB 5 layers architecture",ls="--",linewidth=3,color='orange')

plt.plot(num_epochs, aucs_15lay,label="Signal versus Background 15 layers architecture",linewidth=3,color='magenta')
plt.plot(num_epochs, aucsSB_15lay,label="Background in SR versus SB 15 layers architecture",ls="--",linewidth=3,color='magenta')

plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20, bbox_to_anchor=(1.05, 1.0))
plt.tight_layout()
plt.ylim([0,1])

In [ ]:
class AUCRecordsL9(Callback, ):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"epoch {epoch}")
        
    def on_epoch_end(self, epoch, logs=None):
        global aucs_foravg_9lay, aucsSB_foravg_9lay, aucs_foravg_9lay, aucsSB_foravg_9lay, aucs_9lay, aucs_std_9lay, aucsSB_9lay, aucsSB_std_9lay, num_epochs
        preds_hold_9lay = self.model.predict(norm_func(X_test)[:,1:],batch_size=10000)[:,0]
        preds_holdSB_9lay = self.model.predict(norm_func(X_testSB)[:,1:],batch_size=10000)[:,0]
        num_epochs += [epoch]
        
        myauc_scan_9lay = roc_auc_score(Y_test, preds_hold_9lay)
        print(myauc_scan_9lay)
        myaucSB_scan_9lay = roc_auc_score(Y_testSB, preds_holdSB_9lay)
        print(myaucSB_scan_9lay)
        
        aucs_foravg_9lay += [myauc_scan_9lay]
        print(aucs_foravg_9lay)
        aucsSB_foravg_9lay += [myaucSB_scan_9lay]
        print(aucsSB_foravg_9lay)
        
        aucs_9lay+=[np.mean(aucs_foravg_9lay)]
        print(aucs_9lay)
        aucs_std_9lay += [np.std(aucs_foravg_9lay)]
        print(aucs_std_9lay)
        aucsSB_9lay+=[np.mean(aucsSB_foravg_9lay)]
        print(aucsSB_9lay)
        aucsSB_std_9lay += [np.std(aucsSB_foravg_9lay)]
        print(aucsSB_std_9lay)
        
        print(f"epoch {epoch}")

In [ ]:
    lambdas_9lay = []
    num_epochs = []
    aucs_9lay = []
    aucs_std_9lay = []
    aucsSB_9lay = []
    aucsSB_std_9lay = []
    aucs_foravg_9lay =[]
    aucsSB_foravg_9lay = []

    for k in range(1):
        model_cwola_scan_9lay = Sequential()
        model_cwola_scan_9lay.add(Dense(64, input_dim=4, activation='relu')) 
        model_cwola_scan_9lay.add(Dense(100, activation='relu'))
        model_cwola_scan_9lay.add(Dense(64, activation='relu'))
        model_cwola_scan_9lay.add(Dense(100, activation='relu'))
        model_cwola_scan_9lay.add(Dense(64, activation='relu'))
        model_cwola_scan_9lay.add(Dense(100, activation='relu'))
        model_cwola_scan_9lay.add(Dense(64, activation='relu'))
        model_cwola_scan_9lay.add(Dense(100, activation='relu'))
        model_cwola_scan_9lay.add(Dense(64, activation='relu'))
        model_cwola_scan_9lay.add(Dense(1, activation='sigmoid'))
        model_cwola_scan_9lay.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])
        
        # Generate training datasets
        SR_signal_to_injectT21 = SR_signals_to_injectT21[k]
        SB_signal_to_injectT21 = SB_signals_to_injectT21[k]

        signal_to_testT21 = signals_to_testT21[k]

        X_kcwola_train_9lay = np.concatenate([
            SR_signal_to_injectT21, SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] < SR_low], 
            SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] > SR_high],
            SR_background_dataT21, SB_background_dataT21[SB_background_dataT21[:,0] < SR_low], 
            SB_background_dataT21[SB_background_dataT21[:,0] > SR_high],
            SR_background_simT21, SB_background_simT21[SB_background_simT21[:,0] < SR_low], 
            SB_background_simT21[SB_background_simT21[:,0] > SR_high]
        ])
        Y_kcwola_train_9lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), np.zeros(len(SB_signal_to_injectT21)),
            np.ones(len(SR_background_dataT21)), np.zeros(len(SB_background_dataT21)),
            np.zeros(len(SR_background_simT21)), np.ones(len(SB_background_simT21))
        ])
        
        n_low_data = ((SB_background_dataT21[:,0] > (SB_low)) & ((SB_background_dataT21[:,0] <= SR_low))).sum()
        n_low_sim = ((SB_background_simT21[:,0] > (SB_low)) & ((SB_background_simT21[:,0] <= SR_low))).sum()
        n_low_sig = ((SB_signal_to_injectT21[:,0] > (SB_low)) & ((SB_signal_to_injectT21[:,0] <= SR_low))).sum()

        n_hi_data = ((SB_background_dataT21[:,0] < (SB_high)) & ((SB_background_dataT21[:,0] >= SR_high))).sum()
        n_hi_sim = ((SB_background_simT21[:,0] < (SB_high)) & ((SB_background_simT21[:,0] >= SR_high))).sum()
        n_hi_sig = ((SB_signal_to_injectT21[:,0] < (SB_high)) & ((SB_signal_to_injectT21[:,0] >= SR_high))).sum()

        n_sr_data = len(SR_background_dataT21)
        n_sr_sim = len(SR_background_simT21)
        n_sr_sig = len(SR_signal_to_injectT21)

        w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
        w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)

        w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
        w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
        
        W_kcwola_train_9lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), w_low*np.ones((SB_signal_to_injectT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_signal_to_injectT21[:,0] > SR_high).sum()),
            np.ones(len(SR_background_dataT21)), w_low*np.ones((SB_background_dataT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_background_dataT21[:,0] > SR_high).sum()),
            myklambda*np.ones(len(SR_background_simT21)), 
            w_low_sim*myklambda*np.ones((SB_background_simT21[:,0] <= SR_low).sum()), 
            w_high_sim*myklambda*np.ones((SB_background_simT21[:,0] > SR_high).sum())
        ])
    
        sbmjj = X_kcwola_train_9lay[:,0]
        sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

        X_kcwola_train_9lay = X_kcwola_train_9lay[sb_crit]
        Y_kcwola_train_9lay = Y_kcwola_train_9lay[sb_crit]
        W_kcwola_train_9lay = W_kcwola_train_9lay[sb_crit]


        X_kcwola_train_9lay, Y_kcwola_train_9lay, W_kcwola_train_9lay = shuffle(X_kcwola_train_9lay, Y_kcwola_train_9lay, 
                                                                       W_kcwola_train_9lay)

    
        # Generate testing datasets
        X_test = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
        Y_test = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
        
        X_testSB = np.concatenate([SR_background_data_fortestT21,SB_background_data_fortestT21])
        Y_testSB = np.concatenate([np.ones(len(SR_background_data_fortestT21)),
                                   np.zeros(len(SB_background_data_fortestT21))])

        model_cwola_scan_9lay.fit(norm_func(X_kcwola_train_9lay)[:,1:],Y_kcwola_train_9lay, epochs=EPOCHS, 
                               batch_size=batch_size,validation_data=(norm_func(X_test)[:,1:],Y_test),
                               sample_weight=W_kcwola_train_9lay, verbose=2, callbacks=[AUCRecordsL9()])

        pass


In [ ]:
aucs_9lay = np.asarray(aucs_9lay)
aucs_std_9lay = np.asarray(aucs_std_9lay)

aucsSB_9lay = np.asarray(aucsSB_9lay)
aucsSB_std_9lay = np.asarray(aucsSB_std_9lay)

print(aucs_9lay)
print(aucsSB_9lay)

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_2lay,label="Signal versus Background Model 2 layers architecture",linewidth=3,color='red')
plt.plot(num_epochs, aucsSB_2lay,label="Background in SR versus SB 2 layers architecture",ls="--",linewidth=3,color='red')

plt.plot(num_epochs, aucs_3lay,label="Signal versus Background 3 layers architecture",linewidth=3,color='blue')
plt.plot(num_epochs, aucsSB_3lay,label="Background in SR versus SB 3 layers architecture",ls="--",linewidth=3,color='blue')

plt.plot(num_epochs, aucs_4lay,label="Signal versus Background 4 layers architecture",linewidth=3,color='green')
plt.plot(num_epochs, aucsSB_4lay,label="Background in SR versus SB 4 layers architecture",ls="--",linewidth=3,color='green')

plt.plot(num_epochs, aucs_5lay,label="Signal versus Background 5 layers architecture",linewidth=3,color='orange')
plt.plot(num_epochs, aucsSB_5lay,label="Background in SR versus SB 5 layers architecture",ls="--",linewidth=3,color='orange')

plt.plot(num_epochs, aucs_9lay,label="Signal versus Background 9 layers architecture",linewidth=3,color='cyan')
plt.plot(num_epochs, aucsSB_9lay,label="Background in SR versus SB 9 layers architecture",ls="--",linewidth=3,color='cyan')

plt.plot(num_epochs, aucs_15lay,label="Signal versus Background 15 layers architecture",linewidth=3,color='magenta')
plt.plot(num_epochs, aucsSB_15lay,label="Background in SR versus SB 15 layers architecture",ls="--",linewidth=3,color='magenta')

plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20, bbox_to_anchor=(1.05, 1.0))
plt.tight_layout()
plt.ylim([0,1])

In [ ]:
class AUCRecordsL11(Callback, ):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"epoch {epoch}")
        
    def on_epoch_end(self, epoch, logs=None):
        global aucs_foravg_11lay, aucsSB_foravg_11lay, aucs_foravg_11lay, aucsSB_foravg_11lay, aucs_11lay, aucs_std_11lay, aucsSB_11lay, aucsSB_std_11lay, num_epochs
        preds_hold_11lay = self.model.predict(norm_func(X_test)[:,1:],batch_size=10000)[:,0]
        preds_holdSB_11lay = self.model.predict(norm_func(X_testSB)[:,1:],batch_size=10000)[:,0]
        num_epochs += [epoch]
        
        myauc_scan_11lay = roc_auc_score(Y_test, preds_hold_11lay)
        print(myauc_scan_11lay)
        myaucSB_scan_11lay = roc_auc_score(Y_testSB, preds_holdSB_11lay)
        print(myaucSB_scan_11lay)
        
        aucs_foravg_11lay += [myauc_scan_11lay]
        print(aucs_foravg_11lay)
        aucsSB_foravg_11lay += [myaucSB_scan_11lay]
        print(aucsSB_foravg_11lay)
        
        aucs_11lay+=[np.mean(aucs_foravg_11lay)]
        print(aucs_11lay)
        aucs_std_11lay += [np.std(aucs_foravg_11lay)]
        print(aucs_std_11lay)
        aucsSB_11lay+=[np.mean(aucsSB_foravg_11lay)]
        print(aucsSB_11lay)
        aucsSB_std_11lay += [np.std(aucsSB_foravg_11lay)]
        print(aucsSB_std_11lay)
        
        print(f"epoch {epoch}")

In [ ]:
    lambdas_11lay = []
    num_epochs = []
    aucs_11lay = []
    aucs_std_11lay = []
    aucsSB_11lay = []
    aucsSB_std_11lay = []
    aucs_foravg_11lay =[]
    aucsSB_foravg_11lay = []

    for k in range(1):
        model_cwola_scan_11lay = Sequential()
        model_cwola_scan_11lay.add(Dense(64, input_dim=4, activation='relu')) 
        model_cwola_scan_11lay.add(Dense(100, activation='relu'))
        model_cwola_scan_11lay.add(Dense(64, activation='relu'))
        model_cwola_scan_11lay.add(Dense(100, activation='relu'))
        model_cwola_scan_11lay.add(Dense(64, activation='relu'))
        model_cwola_scan_11lay.add(Dense(100, activation='relu'))
        model_cwola_scan_11lay.add(Dense(64, activation='relu'))
        model_cwola_scan_11lay.add(Dense(100, activation='relu'))
        model_cwola_scan_11lay.add(Dense(64, activation='relu'))
        model_cwola_scan_11lay.add(Dense(100, activation='relu'))
        model_cwola_scan_11lay.add(Dense(64, activation='relu'))
        model_cwola_scan_11lay.add(Dense(1, activation='sigmoid'))
        model_cwola_scan_11lay.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])
        
        # Generate training datasets
        SR_signal_to_injectT21 = SR_signals_to_injectT21[k]
        SB_signal_to_injectT21 = SB_signals_to_injectT21[k]

        signal_to_testT21 = signals_to_testT21[k]

        X_kcwola_train_11lay = np.concatenate([
            SR_signal_to_injectT21, SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] < SR_low], 
            SB_signal_to_injectT21[SB_signal_to_injectT21[:,0] > SR_high],
            SR_background_dataT21, SB_background_dataT21[SB_background_dataT21[:,0] < SR_low], 
            SB_background_dataT21[SB_background_dataT21[:,0] > SR_high],
            SR_background_simT21, SB_background_simT21[SB_background_simT21[:,0] < SR_low], 
            SB_background_simT21[SB_background_simT21[:,0] > SR_high]
        ])
        Y_kcwola_train_11lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), np.zeros(len(SB_signal_to_injectT21)),
            np.ones(len(SR_background_dataT21)), np.zeros(len(SB_background_dataT21)),
            np.zeros(len(SR_background_simT21)), np.ones(len(SB_background_simT21))
        ])
        
        n_low_data = ((SB_background_dataT21[:,0] > (SB_low)) & ((SB_background_dataT21[:,0] <= SR_low))).sum()
        n_low_sim = ((SB_background_simT21[:,0] > (SB_low)) & ((SB_background_simT21[:,0] <= SR_low))).sum()
        n_low_sig = ((SB_signal_to_injectT21[:,0] > (SB_low)) & ((SB_signal_to_injectT21[:,0] <= SR_low))).sum()

        n_hi_data = ((SB_background_dataT21[:,0] < (SB_high)) & ((SB_background_dataT21[:,0] >= SR_high))).sum()
        n_hi_sim = ((SB_background_simT21[:,0] < (SB_high)) & ((SB_background_simT21[:,0] >= SR_high))).sum()
        n_hi_sig = ((SB_signal_to_injectT21[:,0] < (SB_high)) & ((SB_signal_to_injectT21[:,0] >= SR_high))).sum()

        n_sr_data = len(SR_background_dataT21)
        n_sr_sim = len(SR_background_simT21)
        n_sr_sig = len(SR_signal_to_injectT21)

        w_low = 0.5*(n_sr_sig + n_sr_data)/(n_low_sig + n_low_data)
        w_low_sim = 0.5*(n_sr_sim)/(n_low_sim)

        w_high = 0.5*(n_sr_sig + n_sr_data)/(n_hi_sig + n_hi_data)
        w_high_sim = 0.5*(n_sr_sim)/(n_hi_sim)
        
        W_kcwola_train_11lay = np.concatenate([
            np.ones(len(SR_signal_to_injectT21)), w_low*np.ones((SB_signal_to_injectT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_signal_to_injectT21[:,0] > SR_high).sum()),
            np.ones(len(SR_background_dataT21)), w_low*np.ones((SB_background_dataT21[:,0] <= SR_low).sum()), 
            w_high*np.ones((SB_background_dataT21[:,0] > SR_high).sum()),
            myklambda*np.ones(len(SR_background_simT21)), 
            w_low_sim*myklambda*np.ones((SB_background_simT21[:,0] <= SR_low).sum()), 
            w_high_sim*myklambda*np.ones((SB_background_simT21[:,0] > SR_high).sum())
        ])
    
        sbmjj = X_kcwola_train_11lay[:,0]
        sb_crit = ((sbmjj > SR_low - SB_width)&(sbmjj <= SR_high + SB_width))

        X_kcwola_train_11lay = X_kcwola_train_11lay[sb_crit]
        Y_kcwola_train_11lay = Y_kcwola_train_11lay[sb_crit]
        W_kcwola_train_11lay = W_kcwola_train_11lay[sb_crit]


        X_kcwola_train_11lay, Y_kcwola_train_11lay, W_kcwola_train_11lay = shuffle(X_kcwola_train_11lay, Y_kcwola_train_11lay, 
                                                                       W_kcwola_train_11lay)

    
        # Generate testing datasets
        X_test = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
        Y_test = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
        
        X_testSB = np.concatenate([SR_background_data_fortestT21,SB_background_data_fortestT21])
        Y_testSB = np.concatenate([np.ones(len(SR_background_data_fortestT21)),
                                   np.zeros(len(SB_background_data_fortestT21))])

        model_cwola_scan_11lay.fit(norm_func(X_kcwola_train_11lay)[:,1:],Y_kcwola_train_11lay, epochs=EPOCHS, 
                               batch_size=batch_size,validation_data=(norm_func(X_test)[:,1:],Y_test),
                               sample_weight=W_kcwola_train_11lay, verbose=2, callbacks=[AUCRecordsL11()])

        pass


In [ ]:
aucs_11lay = np.asarray(aucs_11lay)
aucs_std_11lay = np.asarray(aucs_std_11lay)

aucsSB_11lay = np.asarray(aucsSB_11lay)
aucsSB_std_11lay = np.asarray(aucsSB_std_11lay)

print(aucs_11lay)
print(aucsSB_11lay)

In [ ]:
fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
ax0 = plt.subplot(gs[0])
ax0.yaxis.set_ticks_position('both')
ax0.xaxis.set_ticks_position('both')
ax0.tick_params(direction="in",which="both")
ax0.minorticks_on()
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.plot(num_epochs, aucs_2lay,label="Signal versus Background Model 2 layers architecture",linewidth=3,color='red')
#plt.plot(num_epochs, aucsSB_2lay,label="Background in SR versus SB 2 layers architecture",ls="--",linewidth=3,color='red')

plt.plot(num_epochs, aucs_3lay,label="Signal versus Background 3 layers architecture",linewidth=3,color='blue')
#plt.plot(num_epochs, aucsSB_3lay,label="Background in SR versus SB 3 layers architecture",ls="--",linewidth=3,color='blue')

plt.plot(num_epochs, aucs_4lay,label="Signal versus Background 4 layers architecture",linewidth=3,color='green')
#plt.plot(num_epochs, aucsSB_4lay,label="Background in SR versus SB 4 layers architecture",ls="--",linewidth=3,color='green')

plt.plot(num_epochs, aucs_5lay,label="Signal versus Background 5 layers architecture",linewidth=3,color='orange')
#plt.plot(num_epochs, aucsSB_5lay,label="Background in SR versus SB 5 layers architecture",ls="--",linewidth=3,color='orange')

plt.plot(num_epochs, aucs_9lay,label="Signal versus Background 9 layers architecture",linewidth=3,color='cyan')
#plt.plot(num_epochs, aucsSB_9lay,label="Background in SR versus SB 9 layers architecture",ls="--",linewidth=3,color='cyan')

plt.plot(num_epochs, aucs_11lay,label="Signal versus Background 11 layers architecture",linewidth=3,color='yellow')
#plt.plot(num_epochs, aucsSB_11lay,label="Background in SR versus SB 11 layers architecture",ls="--",linewidth=3,color='yellow')

#plt.plot(num_epochs, aucs_15lay,label="Signal versus Background 15 layers architecture",linewidth=3,color='magenta')
#plt.plot(num_epochs, aucsSB_15lay,label="Background in SR versus SB 15 layers architecture",ls="--",linewidth=3,color='magenta')



plt.axhline(0.5,linewidth=2, color='gray',ls=":")

plt.xlabel(r'Epochs',fontsize=20)
plt.ylabel(r"AUC",fontsize=20)
plt.legend(frameon=False,fontsize=20, bbox_to_anchor=(1.05, 1.0))
plt.tight_layout()
plt.ylim([0,1])

In [ ]:
from sklearn.model_selection import train_test_split

SR_low = 3.3
SR_high = 3.7
test_size = 0.5
EPOCHS=20
batch_size = 200

def sr_crit(d):
    return (d[:,0] < SR_high) & (d[:,0] >= SR_low)

bg_srsbT21, bg2_srsbT21, sig_srsbT21 = sr_crit(dataset_bgT21), sr_crit(dataset_bg2T21), sr_crit(dataset_sigT21)

SR_background_dataT21 = dataset_bgT21[bg_srsbT21]
SB_background_dataT21 = dataset_bgT21[~bg_srsbT21]

SR_background_simT21 = dataset_bg2T21[bg2_srsbT21]
SB_background_simT21 = dataset_bg2T21[~bg2_srsbT21]

SR_signalT21 = dataset_sigT21[sig_srsbT21]
SB_signalT21 = dataset_sigT21[~sig_srsbT21]

N_inputs = len(SR_background_dataT21.T) - 1


((SR_background_dataT21, SR_background_data_fortestT21),
 (SR_background_simT21, SR_background_sim_fortestT21),
 (SB_background_dataT21, SB_background_data_fortestT21),
 (SB_background_simT21, SB_background_sim_fortestT21),
 ) = [train_test_split(arr, test_size=test_size) for arr in [
    SR_background_dataT21, SR_background_simT21,
    SB_background_dataT21, SB_background_simT21,
]]

mn,mx = np.percentile(np.concatenate([SB_background_dataT21, SB_background_simT21, SR_background_dataT21, 
                                      SR_background_simT21]), [1,99], axis=0)

def norm_func(d):
    return (d - mn)/(mx - mn)

Nsig = 1000
Nsig_SB = int(np.round(len(SB_signalT21)*Nsig/len(SR_signalT21)))

fmt = '{:>20}:  {}'
print(fmt.format('# signal events', Nsig))
print(fmt.format('signal significance', len(SR_signalT21[0:Nsig])/len(SR_background_dataT21)**0.5))
print(fmt.format('s/b ratio', len(SR_signalT21[0:Nsig])/len(SR_background_dataT21)))

n_injectionsT21 = 20
SR_signals_to_injectT21 = [None] * n_injectionsT21
SB_signals_to_injectT21 = [None] * n_injectionsT21
signals_to_testT21 = [None] * n_injectionsT21

for i in range(n_injectionsT21):
    idx = np.isin(range(len(SR_signalT21)), np.random.choice(SR_signalT21.shape[0], size=Nsig, replace=False))
    SR_signals_to_injectT21[i] = SR_signalT21[idx, :]
    signals_to_testT21[i] = SR_signalT21[~idx, :]
    
    idx = np.isin(range(len(SB_signalT21)), np.random.choice(SB_signalT21.shape[0], size=Nsig_SB, replace=False))
    SB_signals_to_injectT21[i] = SB_signalT21[idx, :]

In [ ]:
#CWoLa with T21 and relu 
cwola_rocsT21 = pd.DataFrame(columns=['fpr', 'tpr'])
SB_width = 0.3

for i in tqdm.tqdm(range(n_injectionsT21)):
    
    SR_signal_to_injectT21 = SR_signals_to_injectT21[i]
    SB_signal_to_injectT21 = SB_signals_to_injectT21[i]

    signal_to_testT21 = signals_to_testT21[i]
        
    X_cwola_trainT21 = np.concatenate([SR_signal_to_injectT21, SR_background_dataT21, SB_signal_to_injectT21, SB_background_dataT21])
    Y_cwola_trainT21 = np.concatenate([np.ones(len(SR_signal_to_injectT21)), np.ones(len(SR_background_dataT21)),
                                    np.zeros(len(SB_signal_to_injectT21)), np.zeros(len(SB_background_dataT21))])
    
    X_cwola_trainT21, Y_cwola_trainT21 = shuffle(X_cwola_trainT21, Y_cwola_trainT21, )

    sbmjjT21 = X_cwola_trainT21[:,0]
    sb_critT21 = ((sbmjjT21 > SR_low - SB_width)&(sbmjjT21 <= SR_high + SB_width))
    
    X_cwola_trainT21 = X_cwola_trainT21[sb_critT21]
    Y_cwola_trainT21 = Y_cwola_trainT21[sb_critT21]
    
    w_lowT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] <= SR_low).sum())
    w_highT21 = 0.5*(len(SR_signal_to_injectT21) + len(SR_background_dataT21))/((X_cwola_trainT21[:,0] >= SR_high).sum())

    W_cwola_trainT21 = np.ones_like(Y_cwola_trainT21)
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] <= SR_low)] = w_lowT21
    W_cwola_trainT21[(Y_cwola_trainT21 == 0) & (X_cwola_trainT21[:,0] >= SR_high)] = w_highT21
    
    X_cwola_valT21 = np.concatenate([signal_to_testT21,SR_background_data_fortestT21])
    Y_cwola_valT21 = np.concatenate([np.ones(len(signal_to_testT21)),np.zeros(len(SR_background_data_fortestT21))])
    
    K.clear_session()
    model_cwolaT21 = Sequential()
    model_cwolaT21.add(Dense(64, input_dim=N_inputs, activation='relu')) 
    model_cwolaT21.add(Dense(100, activation='relu'))
    model_cwolaT21.add(Dense(64, activation='relu'))
    model_cwolaT21.add(Dense(100, activation='relu'))
    model_cwolaT21.add(Dense(1, activation='sigmoid'))
    model_cwolaT21.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist_cwolaT21 = model_cwolaT21.fit(X_cwola_trainT21[:,1:], Y_cwola_trainT21, epochs=EPOCHS, 
                                 batch_size=batch_size, verbose=0, sample_weight=W_cwola_trainT21)

    scores_cwolaT21 = model_cwolaT21.predict(X_cwola_valT21[:,1:],batch_size=batch_size)
        
    fpr_cwolaT21, tpr_cwolaT21, _ = roc_curve(Y_cwola_valT21, scores_cwolaT21)
    cwola_rocsT21 = cwola_rocsT21.append({'fpr': fpr_cwolaT21, 'tpr': tpr_cwolaT21}, ignore_index=True)

In [ ]:
 with np.errstate(divide='ignore'):
    tpr_pts = np.linspace(0, 1, 10000)
    
    # CWoLa T21
    fpr_interp = [None] * len(cwola_rocsT21)

    for i, row in cwola_rocsT21.iterrows():
        fpr, tpr = row
        interp = interpolate.interp1d(tpr, fpr, fill_value=float('nan'), bounds_error=False, assume_sorted=True)

        fpr_pts = interp(tpr_pts)
        fpr_interp[i] = fpr_pts

    fpr_interp = np.ma.masked_invalid(1./np.array(fpr_interp))

    cwola_maxT21 = np.nanmax(fpr_interp, axis=0).data
    cwola_minT21 = np.nanmin(fpr_interp, axis=0).data
    cwola_medT21 = np.median(fpr_interp, axis=0).data
    cwola_meanT21 = np.nanmean(fpr_interp, axis=0).data
    cwola_stdT21 = np.nanstd(fpr_interp, axis=0).data
    
    sig_interp = np.ma.masked_invalid(np.sqrt(fpr_interp))
    
    cwola_smeanT21 = np.nanmean(sig_interp, axis=0).data
    cwola_sstdT21 = np.nanstd(sig_interp, axis=0).data
    
    
   

In [ ]:
bands_T21 = {'cwola_T21': (cwola_maxT21, cwola_medT21, cwola_minT21, cwola_meanT21, cwola_stdT21, cwola_smeanT21, cwola_sstdT21, {'label':'$T_{21}$', 'color':'blue'}),}